<h1>Math</h1>

In this notebook, we'll be looking at how to use the Markov Decision Process to solve real problems

To recap what we've done so far and to understand where we are going next, let's quickly summarize the previous notebook

---

<h3>Recap So Far</h3>

So in the previous section, we learned about the Markov Decision Process

So why was this important?

The MDP is important because it gives us a framework for describing the reinforcement learning problem

Without this framework, it's not clear what approach we should take

But using this framework, we can imagine it like the foundation of a house or the structure of a building

Once we have that, we can start to build on top of it

This is better than, say, taking a bunch of sticks off the ground and trying to combine them together randomly to build a house

To build a house that will last for a long time, we need a strong foundation and that requires us to build from a framework

The framework for reinforcement learning is the MDP

In fact, what we will see is that the Bellman equation, which we've already learned, can be used directly to solve our first important problem

That is to say, all we need to do is implement the Bellman equation in Python code and we've already started to do practical work and reinforcement learning

---

OK, so let's start by reviewing the problem that we are trying to solve

This follows directly from what we learned about MDPs

So to start, we have two entities interacting

These two entities are the Agent and the environment

<img src='extras/55.1.PNG' width='500'></img>

The agent is the thing that we are trying to program

The environment is some game or some subset of the world that we want the agent to achieve some goal inside

So how do the agent and the environment interact?

This is entirely described by the states, actions and rewards

At every time step, the agent gets to read some data from the environment

We can think of these like sensors on a robot, like a camera, a temperature sensor, GPS and so forth

We call this the state at the current time $t$, $s_t$ 

At the same time, the agent also receives a reward from the environment We call this $r_t$

And remember, there is no negative or positive connotation associated with the term reward

The reward is not like a cookie we get for behaving well

The reward is just a number

So the state and the reward go from the environment to the agent 

On the other hands, the action goes from the agent to the environment

This is what the agent does in the environment, which has the effect of possibly changing the state

Pressing a button in a video game or turning a steering wheel left or right

So this is our system

We have two components, the agent and the environment

They communicate with each other via signals called states, actions and rewards

Now, what are we trying to do with this system?

We are trying to program the agent to meet some objective

Of course, the term objective is too generic, but also stating specific objectives like win a game of chess or when a game of tic-tac-toe are too specific

We need a general technique that we can apply to all kinds of problems

So what is this technique?

Ultimately, we would like to program the agents such that it maximizes its sum of future rewards

We call this the return

We said that we would like to program our agents such that it maximises the expected return

We say expected return and not just the return, since both the environment and the agents can contain randomness

Expected return means the average return

So it's good to distinguish the reward from the return

We don't just want to maximise the reward because the reward is instantaneous

Trying to maximize the reward would be too short sighted

Instead, we would like to maximise all the rewards in our future

And so how do we accomplish this?

Well, that's what we will discuss in this notebook

But clearly, to solve this problem, we need a way to describe the agents and the environment using math

The language we need is probability

Thus, both the agent and the environment are described using probability distributions

The agent is described by the policy $\pi(a \vert s)$ 

note that this can also be a deterministic function, but when we generalize that, we get a distribution

The environment is described by the state transition distribution $p(s^\prime,r \vert s,a)$

So to recap what our job is, the state transitions should be considered fixed

This is because we don't have any control over the environment

However, we do have control over the policy $\pi$

Therefore, our ultimate goal in reinforcement learning will be to find the best pie to meet our objectives

---

<h3>Notebook online</h3>

OK, so what's the outline for this notebook

We'll see that with this section and the following notebooks, we are always going to follow the same basic pattern 

In reinforcement learning, there are two tasks that we are concerned with

Task number one is called the prediction problem

This is when we are given a policy and we want to evaluate the value of that policy

In other words, we want to answer the question, how good is the given policy ?

Task number two is the control problem

This is when we are given an environment and we want to find the optimal policy for this environment

In other words, we want to answer the question, what is the best policy?

OK, so hopefully that's pretty simple.

Task number one, tell me how good a given policy is

And task number two, tell me what the best policy is

---

In this notebook in particular, we will look at the following methods 

In order to solve task number one, we will employ a method called iterative policy evaluation

We'll see that this is nothing but applying the Bellman equation repeatedly.

This will give us a function where we can pass in a policy and get out its corresponding value 

In order to solve tasks number two, we will study a general approach known as policy improvement

Once we understand the general approach, we will apply that approach in a quite straightforward manner

The first method we will look at is called policy iteration

This will work, but it's not particularly efficient

The second method improves upon policy iteration, and it's called value iteration

---

<h3>The Big Picture</h3>

One important fact that we want to stress about this notebook is this 

We'll be applying the Bellman equation directly

We'll notice that the Bellmon equation depends on the environment dynamics which are encapsulated by the probability distribution $p(s^\prime,r \vert s,a)$

It also depends on the policy, which can also be expressed as a probability distribution $\pi(a \vert s)$

$$\large V_\pi(s) = \sum_a \pi(a \vert s) \sum_{s^\prime} \sum_r p(s^\prime,r \vert s,a) \{r + \gamma V_\pi (s^\prime) \}$$

Now it's reasonable to assume that we can express $\pi(a \vert s)$ as a distribution in code

But what about $p(s^\prime,r \vert s,a)$

We ask ourselves the question, is it reasonable to assume that we know this?

$\large \pi(a \vert s)$ : We know this (because we programmed it)

$\large p(s^\prime,r \vert s,a)$ : Do we know this ?


Imagine we are engineering a self-driving car or we're trying to teach a robot to walk

The data that we read from your sensors will give us the state at any particular time

But do we actually know these state transition distributions?

This would imply that we know what the readings on our sensors will be in the next timestep 

Or if we can't predict them exactly, we at least know their distribution 

In this notebook, we will assume that this is the case

But we should keep this idea in the back of our minds that in reality, this is not something we necessarily know

<h1>Math</h1>

In this section, we'll be studying our first reinforcement learning algorithm

---

<h3>Policy Evaluation</h3>

As mentioned in the introduction, in reinforcement learning will be focused on two main tasks

Task number one is given a policy, tell me how good it is

That is, what is the value function for this policy ?

Task number two is given an environment, tell me what the best policy is?

This section will focus on task number one

In particular, our goal is to find $V(s)$ or $Q(s,a)$, given a policy $\pi(a \vert s)$

$$\large \text{Input : } \pi(a \vert s)$$
 
$$\large \text{Output : } V_\pi(s) \text{ or } Q_\pi(s,a)$$


note that we typically subscript $V$ and $Q$ to make it explicit that the values depend on which policy is being followed

Of course, this must be the case, we expect a good policy to have higher values for $V$ and $Q$ because they yield higher rewards

We expect a bad policy to have lower values of $V$ and $Q$ because they yield the lower rewards

So clearly the reward we get will depend on which policy we are following

---

<h3>Dynamic Programming (DP)</h3>

So the general approach that we will use in this notebook is called Dynamic Programming, or just DP for short

The specific DP algorithm we will discuss in this section is called Policy Evaluation

Again, our goal is given some policy $\pi$ find the value function

We'll start with the state $V(s)$ although the same techniques can be applied to the action value $Q$ as well

To begin, let's recall the Bellman equation

$$\large V_\pi(s) = \sum_a \pi(a \vert s) \sum_{s^\prime} \sum_r p(s^\prime,r \vert s,a) \{r + \gamma V_\pi(s^\prime)\}$$

What we should find interesting about this equation is that there is no need for any special algorithm to solve this problem

That is to say, we can solve this problem without this notebook

If we go through each symbol in this equation, we can recognize that everything is known except for the $V$s

$$\large \color{red}{V_\pi(s)} = \sum_a \color{green}{\pi(a \vert s)} \sum_{s^\prime} \sum_r \color{green}{p(s^\prime,r \vert s,a)} \{\color{green}{r} + \color{green}{\gamma} \color{red}{V_\pi(s^\prime)}\}$$

$\color{green}{known} - \color{red}{Unknown}$

We know $\pi$, so that's just a number

We also know $p(s^\prime,r \vert s,a)$, or at least we are assuming that we do, so that's just a number

The reward $r$ is just a number

And the discount factor $\gamma$ is also just a number

We know that when we multiply one number by another number, it's just another number

So the only unknowns in this equation are the $V$s

Furthermore, this equation is linear in $V$, there are no $V^2$ terms or exponentials, it's just summing and addition

Everything is just a number $\times$ $V$ along with addition

Therefore, this is a linear equation in $V$ 

Now there are multiple Vee's because we have multiple states

---

<h3>Policy Evaluation - Nothing Special</h3>

Suppose that we have $N$ states, then we have $V(s_1),V(s_2),\ldots, V(s_N)$ 

If we were to write down the Bellman equation for each state explicitly and plug in all the values for $\pi$, $p$, $r$ and $\gamma$, we would have N equations and $N$ unknowns

$$\large V_\pi(s_1) = b_1 + c_{11} V_\pi(s_1) + c_{12}V_\pi(s_2) + \ldots + V_{1N}V_\pi(s_N)$$

$$\large V_\pi(s_2) = b_2 + c_{21} V_\pi(s_1) + c_{22}V_\pi(s_2) + \ldots + V_{2N}V_\pi(s_N)$$

$$\ldots$$

$$\large V_\pi(s_N) = b_N + c_{N1} V_\pi(s_1) + c_{N2}V_\pi(s_2) + \ldots + V_{NN}V_\pi(S_N)$$


Therefore, this is just a linear system and we can solve it using regular linear algebra

The problem is this is not scalable in several ways

Firstly, it doesn't handle the case where $N$ is large and secondly, it doesn't handle the case where $p$ is unknown, although neither does dynamic programming

However, dynamic programming does set us up for the subsequent notebooks where we do assume that $P$ is unknown

---

<h3>Finding V(s) Iteratively</h3>

So what is an alternative way for finding the values for $V(s)$? 

As a side note, when we say $V(s)$, we really mean the values of $V(s)$ for all the states, but to avoid having to write down $V(s_1),V(s_2)$ and so on each time, we will just say $V(s)$ for convenience

In any case, let's now move on to an iterative solution for finding $V(s)$

This is the dynamic programming approach

$$\large \text{Initialise: } v_0(s) = 0  \text{or random for all states (0 for terminal s)}$$

$$\large v_{k+1}(s) = \sum_a \pi(a \vert s) \sum_{s^\prime} \sum_r p(s^\prime,r \vert s,a) \left[r + \gamma v_k(s^\prime)\right]$$

So this might be surprising, but this is nothing but simply applying the Bellman equation over and
over again

There is a subtle use of overloaded notation here 

In this case, when we use the $=$ sign, we have to imagine that this is computer code

So we're not really saying that the two sides are equal, although they are, but what we really mean is assign the value of the expression on the right side to the variable on the left side

Also, note that we start by initializing $V(s)$ randomly or to zero

One exception is the terminal state, which always has value zero

We'll also notice that we're now subscripting $V$ with an index $k$

This index $k$ denotes the timestep of our algorithm

So we start at $k = 0$ and every time we update $V(s)$, $k$ increases by one to the next timestep

Now to be clear, this is not a timestep in the environment, but a timestep in our policy evaluation loop

That is, these are timestep is in our code

---

<h3>Why does this work?</h3>

Now, we might wonder, why does this work, why does performing this update again and again until
$k$ approaches infinity lead us to finding $V_\pi(s)$?

The details are beyond the scope of this notebook, but they are closely related to other iterative algorithms in linear algebra

So we can  look those up if we're interested 

Intuitively, we should recognize that the true value that we are looking for $V(\pi \vert s)$ is a fixed point for this update rule (see <strong>Banach fixed point theorem</strong>)

So why is that?

Well, what is a fixed point?

A fixed point means that when we apply this update rule, $V(s)$ no longer changes

So why does $V_\pi(s)$ not change if we apply this update rule?

Well, because this is just Bellman's equation

$$\large V_\pi(s) = \sum_a \pi(a \vert s) \sum_{s^\prime}\sum_{r} p(s^\prime,r \vert s,a) \{r + \gamma V_\pi(s^\prime)\}$$

Bellman's Equation states that these two sides are equal if we plug in $V_\pi(s)$ and that's if we plug in the true values for$V_\pi(s)$

Help : Imagine we already know $V_\pi$, if we ask what is $V_\pi(0)$, we can just look that up, its just a number, well one other much harder way is to calculate it using Bellman's equation, if we use the true $V_\pi$ on the right side we will get also $V_\pi$ on the left side, therefore $V_\pi$ is a fixed point, so Bellman's equation allows us to calculate $V_\pi(s)$ in terms of $V_\pi(s^\prime), s^\prime \text{ represents all the states}$

Therefore, when we have found $V_\pi(s)$, the right side is already equal to the left side and performing this update results in no change

Therefore, we call it a fixed point

So as long as the solution to this system of equations exists, then it will be found by dynamic programming

---

<h3>When do we stop?</h3>

Another important aspect of this algorithm is when to quit

We know that we will approach the true answer as $k$ approaches infinity, but of course, we can't wait an infinite amount of time for the answer

Therefore, we need to have some exit condition

Typically, as is the case with models like these, we exit when our algorithm has converged

We check for convergence by looking at how much $V(s)$ has changed from one iteration to the next

Specifically, let's create a new variable called $\Delta$

$\Delta$ is the maximum change in $V(s)$ over all states during an iteration

$$\large \Delta = \max_s \vert v_{k+1}(s) - v_k(s) \vert $$

Thus, at each iteration of the loop, we check the value of $\Delta$ against some threshold

We get to pick this threshold depending on how accurate we would like our function to be

Once $\Delta$ falls below this threshold, we can consider our algorithm converged and then we can exit

Note that typical values of Delta are decimal numbers, for example, $10^{-3},10^{-5},10^{-8}$ and so forth

---

<h3>Psuedocode</h3>

OK, so let's look at how policy evaluation would be implemented in practice

Note that this is just pseudocode

In real code there are a few details we have yet to discuss, but looking at the pseudocode first will help us understand the principles


$\text{Given: } \pi(a \vert s)$

$\text{Initialise: }V(s) = 0 \text{ or random (except terminal s, where V(s)=0)}$

$Loop:$

$\qquad \Delta = 0$

$\qquad \text{for s in all_non_terminal_states:}$

$\qquad \text{v_old = V(s) \# store the existing value}$

$\qquad V(s) = \sum_a \pi(a \vert s) \sum_{s^\prime}\sum_r p (s^\prime,r \vert s,a) [r + \gamma V(s^\prime)]$

$\qquad \Delta = \max(\Delta, \vert \text{v_old - V(s)} \vert )$

$\text{if }\Delta < \text{threshold:}$

$\qquad break$

First, we accept as input a policy $\pi$

We start by initializing $V(s)$ randomly

Alternatively, it can also be initialized to all zeros

Note that the terminal state should always be zero, so this value should always be initialized to zero and there's no need to update this value since it is known

Next, we enter a loop, let's suppose that this is an infinite loop and we only break out of the loop after our algorithm has converged 

Inside the loop, we initialize Delta to zero.

As we recall, $\Delta$ will store the maximum change in $V(s)$ over the current iteration

Next, we enter a second inner loop that loops over all the states in our environment except for the terminal state

As we recall, the value for the terminal state does not need to be updated since it's always zero

Note that there's an implicit assumption here, which is that we can loop through all the states

So this doesn't handle the case where the state space is too large to loop through, or when the state space is infinite, or when the state space is continuous

For these cases, they'll be discussed in later notebooks

Dynamic programming requires this assumption

OK, so for each state, we'll store the current estimate for $V(s)$ in a variable called $\text{V_old}$ 

This is just a temporary variable to store our old value

Next, we compute the new value for $\text{V(s)}$ using the right side of the Bellman equation 

At this point, we now have our new value for $V(s)$ and we can compare this to our old value $\text{v_old}$

Remember that we would like to store the maximum change in our $\Delta$ variable

Therefore, we can assign $\Delta$ to be the max of current $\Delta$ and the current absolute difference between the new $V(s)$ and $\text{v_old}$

Once the inner loop is complete, we can check $\Delta$ for this iteration

If $\Delta$ is less than our threshold, we can consider our algorithm converged and we can break out of our infinite loop

---

<h3>Implementing the Bellman Equation</h3>

Now, just to be super clear, how do we implement the Bellman equation? 

Note that it has three summations

Well, we should recognize that summations in math are equivalent to loops in code

So in order to break down the Bellman equation into code, we need some loops 

Specifically, since there are three summations, we need three nested for loops

One over the action space for $a$, one over the state space for $s^\prime$ and one over the possible rewards for $r$

$\text{value = 0}$

$\quad \text{for a in all_actions:}$

$\quad \quad \text{for } s^\prime \text{ in all_states:} $

$\quad \quad \quad \text{for r in all_rewards: }$

$\quad \quad \quad \quad value \ += \ \pi(a \vert s)p(s^\prime,r \vert s,a) \left[ r + \gamma V(s^\prime) \right]$

Then we simply accumulate the result in some variable

---

<h3>Implementing Bellman (Deterministic Reward)</h3>

Now, note that the reward is typically deterministic

This is the case for many practical environments and it's also the case for the environments in these following notebooks

Therefore, there is no practical need for the loop over the reward $r$

Instead, we can treat the reward as simply a deterministic function of the state $s^\prime$ 

$\text{value = 0}$

$\quad \text{for a in all_actions:}$

$\quad \quad \text{for } s^\prime \text{ in all_states:} $

$\quad \quad \quad value \ += \ \pi(a \vert s)p(s^\prime,r \vert s,a) \left[ r(s^\prime) + \gamma V(s^\prime) \right]$

In this scenario, we would only need two loops and the reward can simply be a dictionary look up into a reward function that accepts a state as input

---

<h3>Data Structures</h3>

So now that we're getting closer and closer to our Python implementation, it's worth thinking about what kind of data structures we will need 

In our Bellmon update, it's clear that according to this update rule, we need at least two versions of $V$

One which will hold $V$ at iteration $k$ and one which will hold the updated $V$ at iteration $k+1$

So if we use a list, then you would need two lists of the same size if we use a dictionary, then we would need two dictionaries of the same size

And so why don't we need more than two?

Well, suppose that we have two of these lists or dictionaries or arrays or whatever we are using
to hold $V$, let's call them $A$ and $B$

Let's suppose that we've initialised $V(s)$ in a variable called $A$ we'll call that $V_0(s)$

Then we would store the updates in the variable $B$

So now $B$ holds the latest version of $V(s)$, we'll call that $V_1(s)$

But now we don't need the old $A$ anymore so it can hold the next set of updates

And now $A$ will hold the latest version of $V(s)$, we'll call that the $V_2(s)$

So we only need two structures because we can just keep alternating which one holds the latest version of $V(s)$

$\text{Ex: } A = V_0(s), B = V_1(s), A = V_2(s), \ldots$

Now, in practice, what we do is a lot simpler, although technically it goes against this update rule

In practice, we simply store everything in the same structure

We don't bother to have two arrays

Therefore, when we update $V(s)$ at iteration $k+1$, technically it should only depend on values of this four different states at iteration $k$

However, since we store everything in the same structure, $V(s)$ at iteration $k+1$ might depend on other values of $V(s)$ on the same iteration $k+1$

Although this might sound wrong it actually ends up being more efficient

We're encouraged to test it out yourself in the code

And by the way, we call this in-place updating

---

<h3>Representing $V(s)$,$\pi(a \vert s)$ amd $p(s^\prime,r \vert s,a)$</h3>

So, as mentioned, something that's worth thinking about early and often is how you will actually implement these things in code

In math, it's easy to write $V(s)$ and $p(s,r^\prime \vert s,a)$

But what does this actually mean in code?

Let's take something simple like $s$ 

In our grid world environment, each state is a position on the grid

We can think of it like a maze and we're trying to reach the maze exit

So one way to represent each state would be a tuple containing the coordinates on the grid

For example, the top left would be (0,0) to the right of that would be (0,1) and so forth

If our states are tuples, then an appropriate data structure to store $V(s)$ would be a dictionary

In this case, the key to the dictionary is the state represented by a tuple ```int```s

The value of the dictionary is a number representing $V(s)$

```python
V = {(0,0):0.51, (0,1):0.72, ...}
```

<img src='extras/55.2.PNG' width='400'></img>

However, there are other possible approaches

For example, suppose we just numbered each of the states 0,1,2 and so forth

Now our states are just integers and therefore we can use an array or a list instead of a dictionary which is more computationally intensive

E.g. $V = [0.51,0.72,\ldots]$

---

What about the state transitions $p$ 

In this case, we have even more options

Basically, we know that for each four tuple $(s^\prime,r,s,a)$, we're going to have some associated probability value

If states actions and rewards can be represented by a finite set of integers, then we might store $p$ in a four dimensional array

```
p[s',r,s,a] = p(s',r|s,a)
```
But remember that rewards are usually deterministic, so perhaps these can be removed entirely

Another option would be to use a dictionary

The key to this dictionary might be a triple containing $s^\prime,s,a$

As long as the states and actions are hashable, this would be an appropriate dictionary key

```
Key:(s',s,a)
Value: p(s'|s,a)
```

Yet another way to store this distribution would be to use a dictionary with only $s$ as a key 

Then the value of this dictionary might be another nested dictionary with the action $a$ as the key

Then the value of this nested dictionary might be yet another dictionary storing $s^\prime$

Finally, the value of this dictionary might be a tuple containing the associated probability and the corresponding reward $r$

```
probs = {s:{a:{s':p(s'|s,a),...}}}
```

The point of this is when there are so many variables, there are more options to choose from for how we want to implement this data structure and code

So keep this in mind as we go through the notebook and when we wrtie the code

<h1>Math</h1>

In this section, we are going to discuss how to design our reinforcement learning program

---

<h3>Designing our RL Program</h3>

First let's recap what we would normally do in supervised learning 

In supervised learning, we are interested in implementing the algorithm we just learned about 

The main outline is always the same no matter what supervised learning algorithm we're implementing 

The main steps are as follows:

<ol>
    <li>Load in the data</li>
    <li>Instantiate your model</li>
    <li>Train our model</li>
    <li>Evaluate our model</li>
</ol>


Our job as the implementer of the algorithm is to write up the fit and predict functions

This is the case for all supervised learning algorithms

In other words we might want to think of this as a fill in the blanks type of task

Steps 1 to 4 are boilerplate meaning that no matter what algorithm we're implementing you still have to do these things

```python
class MyModel:
    def fit(X,Y):
        # YOUR JOB
    def predict(X):
        # YOUR JOB
        
# boilerplate
Xtrain, Ytrain, Xtest, Ytest = get_data() # 1. Get data
model = MyModel() # 2. Instantiate model
model.fit(Xtrain,Ytrain) # 3. Train model
model.score(Xtest,Ytest) # 4. Evaluate model     
```

---

In this course, since this is reinforcement learning that's not going to be what we'll do

However that doesn't mean there isn't a pattern to be followed

As we did in the bandit notebook, we're going to learn about several different algorithms but the beauty of these algorithms is that like supervised learning they all have the same interface

So whatever our design, we're basically just going to be implementing the same thing multiple times

The only difference should be the algorithm itself not the layout

---

With that said it is still possible to go over the basic steps that our script must perform

First realize that there are two different problems we will try to solve in the following notebooks

So we'll have two different kinds of scripts

The first kind of problem is the prediction problem

This is where, given the policy we would like to find the corresponding value function $V$ or $Q$

The second kind of problem is the control problem

This is where our agent will engage with its environment and do actual learning

The goal of the second problem is to find the optimal policy and the optimal corresponding value function

---

<h3>Prediction Problem</h3>

The first problem, the prediction problem is the easier of the two

Our job is essentially to do one single thing, find the value function 

The algorithms we discuss in this notebook are iterative, so the basic outline will be like this

$\text{Goal: Find V(s)}$

```python
given: policy
V(s) = initial value
for t in range(max_iterations):
    states,actions,rewards = play_game(policy)
    update V(s) given (states,actions,rewards) using the algorithm we learned
print useful info (change in V(s) vs time, final V(s), policy)

```

First we initialize $V(s)$ 

Then in a loop, we play the game according to the given policy 

From this playing of the game, we collect data about the states, actions and rewards 

Then we use those states actions and rewards to update our value function $V(s)$ according to whatever algorithm we just learned about

At the end we might want to plot or print some useful information such as the change in $V(s)$ per iteration, the final values of $V(s)$ and the actual policy itself so that we can ensure that the values we found make sense

---

<h3>Control Problem</h3>

The second type of problem the control problem is a bit more difficult because it requires us to update two things at the same time, the policy and the value function 

In this case we're not given a policy, but rather our goal is to update the policy according to whatever algorithm we learn

```python
initialise value function and policy
for t in range(max_iterations):
    states, actions, rewards = play_game(policy)
    update value function and policy according to (states, actions, rewards) using the algorithm we learned
print useful info (change in V(s) vs. time, final V(s), final policy)

```

Still the basic loop is the same

First we initialize the value function that can be $V$ or $Q$ depending on the algorithm being discussed

Then we enter a loop for a certain number of iterations 

Inside the loop, we play an episode of the game using the current policy 

From this we get a series of states, actions and rewards that the agent experienced

Then we update the value function and the policy using whatever algorithm we just learn about 

At the end, we again might want a plot or print some useful information such as the change in the value function per iteration, the final values of the value function and the final policy so that we can verify that it makes sense

Now we want to make it clear that this is just a very rough outline 

In actuality for some algorithms the policy is not explicitly represented in code and so it's not actually going to be represented by a python
variable but again that's kind of part of the implementation, so it's partly our choice about how you want to do things

<h1>Math</h1>

In this section, we are going to start looking at the code for the Gridworld environment

This is in preparation for implementing algorithms such as policy evaluation and policy iteration

Obviously, in order to implement these algorithms, we need to have an environment to implement them on

So our first step will be to actually implement Gridworld

---

<h3>Gridworld in Code</h3>

It's important to note that before we start, there are an infinite number of ways to do this, many of them are drastically different

That's why it's so important for us to code by ourselves and implement our own design

The instructor's design is merely one of many, and it's very possible that what we have in our head will be completely different from the instructor's

What we end up designing is very dependent on our personality, our past life experiences and our general education background

Probably someone who is a biologist will write code that's very different from someone who is a computer scientist

And even still, one computer scientist will write code that's very different from another computer scientist

So don't get stuck on trying to make our code like the instructor's code or trying to understand why the intructor's code isn't more like our code

This is completely missing the point of these exercises

Instead, it's better just to do everything completely using our own design from the beginning and to write everything in our own style

What we present in the following lectures is just an example

So let's begin

---

<h3>How will we use it?</h3>

OK, so before even jumping into the code, we want to think about how we will use the code

This is similar to test driven development

The goal is to have some kind of object to represent our environment, a Gridworld object 

In the constructor for the environment, it might make sense to have a few arguments

For example, the number of rows in the grid, the number of columns in the grid and the start position

We might want to have a function that returns the current state

This basically answers the question, in what position are we in at this current moment?

As we've seen, the agent and the environment interact in a loop where the agent performs an action in the environment, arrives in the next state and received some reward

Thus, we propose a function called ```move``` that takes in an action, does the action in the environment and then returns the reward associated with doing that action and arriving in the next state

Obviously, if we want to know what state we're in after taking the action, we simply need to call the function ```current_state```, which we just described

Of course, we can't loop forever, we must know when the game is over

So we propose a function called ```game_over```, which returns true when we are in a terminal state and false otherwise

All right, so we hope this seems pretty simple so far

```python
g = Gridworld(rows, cols, start_position)

# returns current state
g.current_state()

# return reward
reward = g.move(action)

# returns true if in terminal state
g.game_over()

# use it like this

g = Gridworld(rows,cols, start)
while not g.game_over():
    s = g.current_state()
    a = policy(s)
    r = g.move(a)


```

---

<h3>Specifying Rewards and Actions</h3>

Of course, we have yet to discuss one crucial detail

How do we specify what the rewards are?

How do we specify states like walls where the agent cannot travel to?

Now, again, this is very implementation specific, but here's what we will find in the repository (instructor's code on github LazyProgrammer)

We have a function called ```set``` that accepts as input a dictionary containing rewards and also a dictionary of actions that result in movement in the grid world environment

An example of the rewards dictionary would be as follows

```python
rewards = {(0,3):1, (1,3):-1}
```

<img src='extras/55.3.PNG' width='400'></img>

A key $(0,3)$ with a corresponding value of $+1$, means that we will get $+1$ reward when we land in the state $(0,3)$

<img src='extras/55.4.PNG' width='400'></img>

A key $(1,3)$ with the corresponding value of $-1$ means that we will get $-1$ reward when we land in the state $(1,3)$

<img src='extras/55.5.PNG' width='400'></img>

A dictionary of actions can be specified as follows

```python
actions = {
    (0,0) : ('D','R'),
    (0,1) : ('L','R'),
    ...
}
g.set(rewards,actions)
```

Let's focus on a few particular states so you get the idea

<img src='extras/55.3.PNG' width='400'></img>

Notice how the values for $0,0$ are ```D``` and ```R```, which stands for down and right

That's because in the position $(0,0)$, only the actions down and right can result in us going to a different state

Note that it's possible to use the action left and up, they simply don't result in going to a different state, it's the equivalent of walking into a wall

Similarly, if we look at the values for $(0,1)$, we have only left and right

It's not possible to go down because there is a wall in that position

So if we enter the action down, we will simply be walking into a wall and we will remain in the
same state

Finally, notice that the position $(0,3)$ and $(1,3)$ do not appear in this dictionary at all

Remember that these are terminal states, so no actions can be done from these states that would result in going to a different state

---

<h3>Recap: Using the Gridworld class</h3>

So here's an example of how we might use our Gridworld class

```python
g = Grid(3,4,(2,0))
rewards = {(0,3):1,(1,3):-1}
actions = {
    (0,0): ('D','R'),
    (0,1): ('L','R'),
    ...
}

g.set(rewards,actions)

# play the game
while not g.game_over():
    s = g.current_state()
    a = policy(s)
    r = g.move(a)
```

First, we instantiate an object of type Gridworld

Then we call the set function passing in the rewards dictionary and the actions dictionary that we just described

Then we can actually play the game with our agent using the ```move``` function described earlier

---

<h3>grid_world.py</h3>

All right, so now that we've described how we will use the Gridworld class, let's have a look at the actual implementation

```python
class Grid: # Enviroment
    def __init__(self,rows,cols,start):
        self.rows = rows
        self.cols = cols
        self.i = start[0]
        self.j = start[1]
```

So here's the constructor 

As promised, it accepts as input three arguments

The number of rows of the grid, the number of columns of the grid and the starting position

All we do in the constructor is assign these to instance variables ```self.rows```, ```self.cols``` and ```self.i``` and ```self.j```

---

```python
    def set(self,rewards,actions):
        # reward = dict of: (i,j):r (row,col): reward
        # actions = dict of: (i,j): A ()row,col: action list
        self.rewards = rewards
        self.actions = actions
```

Next, we have the set function that takes in a dictionary of rewards and actions

Again, all we do is assign these two instance variables

---

```python
    def set_state(self,s):
        self.i = s[0]
        self.j = s[1]
```

Next, we have the set state function, which will be useful in later code examples

All we do in this function is take as input a state $s$ and assign that to the instance variables ```self.i``` and ```self.j```

Note that this is a bit like playing a video game with cheets 

Obviously in real life games, we can't simply set the state to whatever we want

For example, if we're playing chess, we can't say we want to go to the state where we're one move away from a checkmate

That's simply not allowable when we are playing chess

---

```python
    def current_state(self):
        return (self.i,self.j)
```

Next, we have the current state function, which returns the current state as a tuple containing the $i$ coordinate and the $j$ coordinate

---

```python
    def is_terminal(self,s):
        return s not in self.actions
```

Next, we have the ```is_terminal``` function, which accepts as input a state $s$ and tells us whether or not that state is a terminal state

Roughly speaking, a terminal state is a state we can't move from, although there are subtleties there that we don't need to concern ourselves with

Alternatively, we could have simply said terminal states explicitly, but we think it's reasonable to simply say that any state that does not appear in the actions dictionary is a terminal state

---

```python
def get_next_state(self,s,a):
    i,j = s[0],s[1]
    if a in self.actions[(i,j)]:
        if a == 'U':
            i -= 1
        elif a == 'D':
            i += 1
        elif a == 'R':
            j += 1
        elif a == 'L':
            j -= 1
    return i,j
```

Next, we have the ```get_next_state``` function, which will be useful later on

This accepts as input a state $s$ and an action $a$ and tells us what the next day we end up in will be

This is, again, one of those cheat mode like features, and it only makes sense in this particular environment because this environment is assumed to be deterministic

When we are in a state RsR and you perform an action $a$, we will always end up in the same next state

So first we extract the $i$ and $j$ coordinates from the $s$ variable

Next we check if the action $a$ is in the actions dictionary for the state $(i,j)$

If it's not, then we can simply return the tuple $(i,j)$ since any action from the state won't change the state 

Otherwise, if the action is in the dictionary for this state, then we check what the action is

It's important to remember the convention when we think about arrays and programming, rows count down and the columns count left to right

So if the action is up, we subtract one from the row coordinate

If the action is down, we add one to the row coordinate

If the action is left, we subtract one from the column coordinate.

And if the action is right, we add one to the column coordinate

---

```python
    def move(self,action):
        # check if legal move first
        if action in self.actions[(self.i,self.j)]:
            if action == 'U':
                self.i -=1
            elif action == 'D':
                self.i += 1
            elif action == 'R':
                self.j += 1
            elif action == 'L':
                self.j -=1
        return self.rewards.get((self.i,self.j),0)
    
```

Next, we have the ```move``` function

This is very similar to the get next state function, except that it actually performs the action in the environment rather than being simply a hypothetical calculation

Notice how we use ```self.i``` and ```self.j```, the actual current position

We do the same calculation as discussed previously to update the state, and then at the end, we return the reward associated with the resulting state

This is the case even if we end up in the same state we started in

Note that not all states have associated rewards

So if a state doesn't exist in the rewards dictionary, we will assume that by default the reward is zero

---

```python
    def undo_move(self,action):
        if action == 'U':
            self.i += 1
        elif action == 'D':
            self.i -=1
        elif action == 'R':
            self.j -= 1
        elif action == 'L':
            self.j += 1
        # should never happen
        assert(self.currenst_state() in self.all_states())
```

Next, we have the ```undo_move``` function 

Again, we didn't discuss this before, since it's not critical to playing a game, but it may or may not be useful later on in the following notebooks

As we can see, this simply reverses the calculation done inside the ```move``` function

So all the updates for ```i``` and ```j``` are reversed.

Note that in order to ensure that we don't end up in an illegal state, we have an assert at the end of this function

We assert that the current state is in the set of all states

We will look at the ```all_states``` function very shortly

---

```python
def game_over(self):
    # returns true if game is over, else false
    # true if in a state where no actions are possible
    return (self.i,self.j) not in self.actions
```

Next, we have the game over function

This tells us whether we are in a terminal state currently, which means the game is over

This is slightly different from the ```is_terminal``` function, which checks whether the state we pass in as input is a terminal state

---

```python
    def all_states(self):
        # possibly buggy but simple way to get all states
        # either a position that has possible next actions
        # or a position that yields a reward
        return set(self.action.keys() | set(self.rewards.keys()))
```

Finally, we have the all states function 

Again, there are probably many ways to do this, but this was just a quick and dirty way to get all of the possible states, including the terminal states, as a single set 

To calculate this, we get the set of all states that appear in the actions dictionary and the set of all states that appear in the rewards dictionary

We then return the union of these two sets

The reason we want to do this is some states do not appear in actions dictionary such as terminal states

Conversely, there are some states that don't appear in the rewards dictionary since there will be some states that don't yield any non-zero reward

Obviously this calculation isn't fullproof, but it's probably good enough

---

```python
    def standrad_grid():
        g = Grid(3,4,(2,0))
        rewards = {(0,3):1,(1,3):-1}
        actions = {
            (0,0): ('D','R'),
            (0,1): ('L','R'),
            (0,2): ('L','D','R'),
            (1,0): ('U','D'),
            (1,2): ('U','D','R'),
            (2,0): ('U','R'),
            (2,1): ('L','R'),
            (2,2): ('L','R','U'),
            (2,3): ('L','U')
        }
        g.set(rewards,actions)
        return g
```

Finally, note that we have a helper function called ```standard_grid```, which does what we described earlier

It creates a $3 \times 4$ grid of the exact same type we have been discussing in the Math sections 

It instantiates a rewards dictionary and an action dictionary and then sets these on the grid and returns the Gridworld object

<h1>Math</h1>

In this next section, we are going to look at the code for iterative policy evaluation

We'll be using the deterministic version of GridWorld, which we just described

---

<h3>Iterative Policy Evaluation in Code</h3>

Again, before we start, we want to discuss at a high level what we are going to do before we dive into the implementation

This is similar to a test driven development approach where we define what we want to implement before actually implementing it

Another way to think about this is that this is as close as we will get to a fill in the blanks type of exercise

In fact, we are defining where the blanks are

So let's begin

---

<h3>Visualise the value and policy</h3>

First, we would like to have some functions to visualize the value and the policy

We might have a function called the print values, which takes in a value table $V$ and a Gridworld object $G$ and prints the value of each state on top of a drawing of the environment

We might have a function called print policy, which takes in a policy table $P$ and a Gridworld object $G$ and prints the action corresponding to each state on top of a drawing of the environment

<img src='extras/55.6.PNG' width='700'></img>

---

<h3>State transition probabilities</h3>

Next, because this is dynamic programming, we know that this algorithm involves using the state transition probabilities to calculate $V(s)$

But our grid world environment does not have any such state transition probabilities

So this is kind of an unusual step, but it's necessary in order to complete the exercise

We are going to build the state transition probabilities from the environment ("somehow")

We'll store the environment dynamics in a dictionary called transition probs

In the most general case, we would have four arguments into our probability function, $s,a,s^\prime,r$, however, to make things a little simpler we will assume that the reward is deterministic so that we only need three arguments into our probability function $s,a,s^\prime$, ```probs[(s,a,s',r)] == p(s',r|s,a)```

Therefore, the keys to the transition probs dictionary will be a triple containing the current state $s$, action $a$, and next state $s^\prime$

The value of the dictionary will obviously be the actual probability $p(s^\prime \vert s,a)$ ```probs[(s,a,s')] == p(s'|s,a)```

How we actually populate this dictionary is an implementation detail, so we'll discuss that later

---

<h3>Policy</h3>

Next, because this is policy evaluation, we need a policy to evaluate 

We'll represent our policy as a dictionary with the key being the state and the value being the action for that state

For example, Up, Down, Left or Right

---

<h3>Run policy evaluation loop</h3>

Next, we will initialise our value table and then run our policy evaluation loop 

At some point we'll also want to print the policy along with the value using the functions we defined earlier

<ul>
    <li>Initialise $V(s)=0$ for all $s$</li>
    <li>Find $V(s)$ given the policy</li>
    <li>Print the policy/value</li>
</ul>

---

<h3>Overall steps</h3>

So overall, these are the steps

<ul>
    <li>Define helper functions to print the policy and the value</li>
    <li>Create dictionaries to represent the state transition probabilities and the policy</li>
    <li>Apply the iterative policy evaluation algorithm to find the value for the given policy</li>
    <li>Call the print policy and print value functions to observe that the results make sense</li>
</ul>

<h1>code</h1>

In [1]:
import numpy as np

In [2]:
# first lets implement our Gridworld enviroment
# we will be using the same interface as described above

class GridWorld:
    def __init__(self,rows,cols,start_pos):
        self.rows = rows
        self.cols = cols
        self.pos = start_pos
    
    def current_state(self):
        return self.pos
    
    def set_state(self,state):
        self.pos = state
        
    def set_actions_rewards(self,actions,rewards):
        self.actions = actions
        self.rewards = rewards
        
    def all_states(self,):
        return [(i,j) for i in range(self.rows) for j in range(self.cols)]
    
    def is_terminal(self,state):
        # we cant perform an action in a terminal state
        # so we expect not to find it in the keys
        return state not in self.actions.keys()
    
    def game_over(self):
        return self.is_terminal(self.pos)
    
    def next_state(self,s,a):
        # return next state should we take action a
        # but do not take that action
        i,j = s
        # if we are allowed to do this action in our current position
        if a in self.actions.get(s,[]): 
            if a == 'R':
                j += 1
            if a == 'L':
                j -= 1
            if a == 'U':
                i -= 1
            if a == 'D':
                i += 1
        return (i,j)
    
    def move(self,a):
        # return next state should we take action a
        # this time we actually make the move
        # and thus return reward
        i,j = self.pos
        # if we are allowed to do this action in our current position
        if a in self.actions.get(self.pos,[]): 
            if a == 'R':
                j += 1
            if a == 'L':
                j -= 1
            if a == 'U':
                i -= 1
            if a == 'D':
                i += 1
        self.pos = (i,j)
        # if no reward return 0
        return self.rewards.get(self.pos,0)
    
    # lets add methods to print values and actions
    def print_values(self,V):
        for i in range(self.rows):
            print('-'*9*self.rows)
            for j in range(self.cols):
                v = V[i,j]
                if v >= 0:
                    # if 0 or +ve, add space so it aligns well when we have -
                    print(" %.2f|" %v,end="")
                else:
                    print("%.2f|" %v,end="")
            print("")
    
    def print_policy(self,P):
        for i in range(self.rows):
            print('-'*6*self.rows)
            for j in range(self.cols):
                print(" %s |" %P.get((i,j),' '),end="")
            print("")
                
    def state_to_loc(self,state): # convert state tupe (0,1) to location 1
        return state[0]*self.rows + state[1]
            

In [3]:
def standrad_grid():
    g = GridWorld(3,4,(2,0))
    rewards = {(0,3):1,(1,3):-1}
    actions = {
        (0,0): ('D','R'),
        (0,1): ('L','R'),
        (0,2): ('L','D','R'),
        (1,0): ('U','D'),
        (1,2): ('U','D','R'),
        (2,0): ('U','R'),
        (2,1): ('L','R'),
        (2,2): ('L','R','U'),
        (2,3): ('L','U')
    }
    g.set_actions_rewards(actions,rewards)
    return g

In [4]:
g = standrad_grid()

In [5]:
# now lets build the transition probabilities
def build_transition_probs(g):
    # we want to calculate p(s'|s,a)
    # if there is an action a to take us from s to s'
    # then p(s' | s,a) is 1
    # else it is 0
    # recall that our enviroment is deterministic
    # for each action, there is only one state to go to
    all_states = g.all_states()
    all_actions = ['U','R','D','L']
    probs_env = {}
    for s in all_states:
        for a in all_actions:
            s_prime = g.next_state(s,a)
            probs_env[(s_prime,s,a)] = 1
    
    return probs_env

In [6]:
p = build_transition_probs(g)

In [7]:
# fixed policy (deterministic)
policy = {
    (2,0) : 'U',
    (1,0) : 'U',
    (0,0) : 'R',
    (0,1) : 'R',
    (0,2) : 'R',
    (1,2) : 'U',
    (2,1) : 'R',
    (2,2) : 'U',
    (2,3) : 'L'    
}

In [8]:
g.print_policy(policy)

------------------
 R | R | R |   |
------------------
 U |   | U |   |
------------------
 U | R | U | L |


In [9]:
# build pi(a|s)
def build_action_probs(policy):
    all_states = g.all_states()
    state_space = len(all_states)
    pi = {}
    for s in all_states:
        if not g.is_terminal(s):
            a = policy.get(s)
            pi[a,s] = 1
    return pi

In [10]:
pi = build_action_probs(policy)

In [11]:
# now we perform policy evaluation
def policy_eval(g):
    all_states = g.all_states()
    V = np.zeros((g.rows,g.cols))
    all_actions = ['U','R','D','L']
    delta = 0
    thresh = 1e-5
    gamma = 0.9
    
    i = 1
    while True:
        # this is just to calculate delta
        V_old = V.copy()
        for s in all_states:
            v_new = 0
            for a in all_actions:
                for s_prime in all_states:
                    v_new += pi.get((a,s),0)*p.get((s_prime,s,a),0)*(g.rewards.get(s_prime,0) + gamma*V[s_prime])
            V[s] = v_new
        delta = np.max(np.abs(V_old-V))
        print('iteration: ',i,' delta: ',delta)
        print('Values: ')
        g.print_values(V)
        print('')
        if delta < thresh:
            break
        i += 1
        V_old = V.copy()
    return V             

In [12]:
V = policy_eval(g)

iteration:  1  delta:  1.0
Values: 
---------------------------
 0.00| 0.00| 1.00| 0.00|
---------------------------
 0.00| 0.00| 0.90| 0.00|
---------------------------
 0.00| 0.00| 0.81| 0.73|

iteration:  2  delta:  0.9
Values: 
---------------------------
 0.00| 0.90| 1.00| 0.00|
---------------------------
 0.00| 0.00| 0.90| 0.00|
---------------------------
 0.00| 0.73| 0.81| 0.73|

iteration:  3  delta:  0.81
Values: 
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.90| 0.00|
---------------------------
 0.66| 0.73| 0.81| 0.73|

iteration:  4  delta:  0.0
Values: 
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.90| 0.00|
---------------------------
 0.66| 0.73| 0.81| 0.73|



In [13]:
# our algorithm converges quite fast
# we can see that the values make sense 
# even if we dont check them by hand (feel free to do so)
# of course the value right beside the winning state is just 1
# since a reward of 1 is obtained at the winning state
# one step away from that is 0.9, since gamma is 0.9
# one step away from 0.9 is 0.81, since 0.9 x 0.9 = 0.81
# and so on

<h1>Math</h1>

In this section, we are going to look at a slightly more complex version of grid world called Windi Gridworld

---

<h3>Windy Gridworld in code</h3>

Previously, our Gridworld was deterministic, meaning all of the transition probabilities were either zero or one 

In  Windy Gridworld, we extend this idea so that the transition probabilities can be anything

Unfortunately, this code is quite a bit messier than the deterministic Gridworld code, since we have to specify all the probabilities

Let's start again with a kind of test driven development approach where we discuss how we would like the code to work before looking at the actual implementation

---

<h3>Representing transition probabilities</h3>

So for windy Gridworld, the main new data structure we want is something to represent the transition probabilities

Again, this will vary widely depending on our personal design choices, but the instructor's design was to represent the probabilities as a dictionary

The dictionary is structured as follows

Similar to our previous code, we've decided that only the next states may be probabilistic

The rewards will be a deterministic function of the state

Therefore, we only need to concern ourselves with the current state, action and the next state

The key to the dictionary will be a tuple of state and action

These represent $s$ and $a$. ```key:(s,a)``` 

The value in the dictionary will be another dictionary containing the next state probabilities 

Inside this nested dictionary, the key will be the possible next state and the value will be the probability of going there, ```value:{s':prob}```

For example, a dictionary with the key ```(0,2)``` and value ```0.7``` means that we will go to state ```(0,2)``` with probability ```0.7```    
 

If there's another key ```(1,3)``` with the value ```0.3```, that means we will go to state ```(1,3)``` with probability ```0.3```

```python
probs = {
    ((2,0),'U') : {(1,0):1.0},
    ((1,2),'U'):{(0,2):0.7,(1,3):0.3}
}
```

Of course, these probabilities must sum to one in order to represent a valid distribution

Note that we can also have only a single possible next state with probability one

This just means that the transition is deterministic

---

In our version of Windy Gridworld, the list of probabilities is quite long, but we are encouraged to look at it carefully in case the instructor made any errors (instructor's request)

Creating this dictionary is repetitive work, so errors are very easy to make

We'll notice that most of the grid is still deterministic

It's just represented in probabilistic form

```python
probs = {
    ((2,0),'U'):{(1,0):1.0},
    ((2,0),'D'):{(2,0):1.0},
    ...
    ((1,2),'U'): {(0,2):0.5, (1,3):0.5}
}
```

The relevant part of this dictionary, which is what makes the windedness interesting, is the state right beside the losing state 

For this state, we obviously want to go up since that brings us closer to the winning state

However, we've made the transition probabilistic so that even if our action is up, we still have
some probability of going to the right

This makes it so that going in this direction is less safe than simply going around along the left wall to get to the goal state

If we recall, the path from the starting position to the goal state is the same whether we go up, up, right, right , right or  right, right, up, up, right 

---

<h3>Instantiating Windy Gridworld</h3>

To continue on with our test driven approach, the way we would like to use our dictionary of transition probabilities is like this

It's very similar to before where we instantiate a windy Gridworld object with the number of rows and columns and the start position 

In the set function, instead of just the rewards dictionary and the actions dictionary, we also pass in the state transition probabilities

```python
g = WindyGrid(3,4,(2,0))
rewards = {(0,3):1,(1,3):-1}
actions = {...}
probs = {...}
g.set(rewards,actions,probs)
```

---

<h3>Other functions</h3>

Other than this, our API should be the same as before

We still want to have a move function which takes in an action and returns a reward

We still want a current state function to return the current state

We still want and is terminal function, a game over function and an all states function, which we've seen before, so you know how these might be used

One function we will not have is the ```get_next_state``` function

As we recall, this takes in a current state $s$, action $a$ and returns the next state that doing this action will land us in

As a quiz question, think about why we will not have this function for Windy Gridworld ?

```python
g = WindyGrid(rows,cols,start_position)
g.move(action)
g.current_state()
g.is_terminal(state)
g.game_over()
g.all_states()

# why not g.next_state(state,action)?
```

---

<h3>get_next_state(state,action)</h3>

So the reason we won't have a function called ```get_next_state``` for windy Gridworld is because having a function like this doesn't make sense any longer

The next state is probabilistic, given a state and an action

Therefore, we can't possibly return a single next state

Now that you understand how we will use Wendy Grid World, let's dive into the implementation

<h1>code</h1>

In [2]:
import numpy as np

In [3]:
class WindyGridWorld:
    def __init__(self,rows,cols,start_pos):
        self.rows = rows
        self.cols = cols
        self.pos = start_pos
    
    def current_state(self):
        return self.pos
    
    def set_state(self,state):
        self.pos = state
        
    def set_actions_rewards_probs(self,actions,rewards,probs):
        self.actions = actions
        self.rewards = rewards
        # we need probs to know where to go after taking action
        self.probs = probs
        
    def all_states(self,):
        return [(i,j) for i in range(self.rows) for j in range(self.cols)]
    
    def is_terminal(self,state):
        # we cant perform an action in a terminal state
        # so we expect not to find it in the keys
        return state not in self.actions.keys()
    
    def game_over(self):
        return self.is_terminal(self.pos)
    
    def move(self,a):
        # this time we sample a next state
        next_state_2_probs = self.probs(self.pos,a)
        next_states = list(next_state_2_probs.keys())
        probs = list(next_state_2_probs.values())
        next_state = np.random.choice(next_states,p=probs)
        self.pos = next_state
        return self.rewards.get(self.pos,0)
    
    # lets add methods to print values and actions
    def print_values(self,V):
        for i in range(self.rows):
            print('-'*9*self.rows)
            for j in range(self.cols):
                v = V[i,j]
                if v >= 0:
                    # if 0 or +ve, add space so it aligns well when we have -
                    print(" %.2f|" %v,end="")
                else:
                    print("%.2f|" %v,end="")
            print("")
    
    def print_policy(self,P):
        for i in range(self.rows):
            print('-'*6*self.rows)
            for j in range(self.cols):
                print(" %s |" %P.get((i,j),' '),end="")
            print("")
                
    def state_to_loc(self,state): # convert state tupe (0,1) to location 1
        return state[0]*self.rows + state[1]


In [4]:
def windy_grid():
    g = WindyGridWorld(3,4,(2,0))
    rewards = {(0,3):1,(1,3):-1}
    actions = {
        (0,0): ('D','R'),
        (0,1): ('L','R'),
        (0,2): ('L','D','R'),
        (1,0): ('U','D'),
        (1,2): ('U','D','R'),
        (2,0): ('U','R'),
        (2,1): ('L','R'),
        (2,2): ('L','R','U'),
        (2,3): ('L','U')
    }
    probs = {
        ((2, 0), 'U'): {(1, 0): 1.0},
        ((2, 0), 'D'): {(2, 0): 1.0},
        ((2, 0), 'L'): {(2, 0): 1.0},
        ((2, 0), 'R'): {(2, 1): 1.0},
        ((1, 0), 'U'): {(0, 0): 1.0},
        ((1, 0), 'D'): {(2, 0): 1.0},
        ((1, 0), 'L'): {(1, 0): 1.0},
        ((1, 0), 'R'): {(1, 0): 1.0},
        ((0, 0), 'U'): {(0, 0): 1.0},
        ((0, 0), 'D'): {(1, 0): 1.0},
        ((0, 0), 'L'): {(0, 0): 1.0},
        ((0, 0), 'R'): {(0, 1): 1.0},
        ((0, 1), 'U'): {(0, 1): 1.0},
        ((0, 1), 'D'): {(0, 1): 1.0},
        ((0, 1), 'L'): {(0, 0): 1.0},
        ((0, 1), 'R'): {(0, 2): 1.0},
        ((0, 2), 'U'): {(0, 2): 1.0},
        ((0, 2), 'D'): {(1, 2): 1.0},
        ((0, 2), 'L'): {(0, 1): 1.0},
        ((0, 2), 'R'): {(0, 3): 1.0},
        ((2, 1), 'U'): {(2, 1): 1.0},
        ((2, 1), 'D'): {(2, 1): 1.0},
        ((2, 1), 'L'): {(2, 0): 1.0},
        ((2, 1), 'R'): {(2, 2): 1.0},
        ((2, 2), 'U'): {(1, 2): 1.0},
        ((2, 2), 'D'): {(2, 2): 1.0},
        ((2, 2), 'L'): {(2, 1): 1.0},
        ((2, 2), 'R'): {(2, 3): 1.0},
        ((2, 3), 'U'): {(1, 3): 1.0},
        ((2, 3), 'D'): {(2, 3): 1.0},
        ((2, 3), 'L'): {(2, 2): 1.0},
        ((2, 3), 'R'): {(2, 3): 1.0},
        ((1, 2), 'U'): {(0, 2): 0.5, (1, 3): 0.5},
        ((1, 2), 'D'): {(2, 2): 1.0},
        ((1, 2), 'L'): {(1, 2): 1.0},
        ((1, 2), 'R'): {(1, 3): 1.0},
      }
    g.set_actions_rewards_probs(actions,rewards,probs)
    return g


In [5]:
g = windy_grid()

In [6]:
# another change is that we will introduce a probabilistic policy
# recall that starting at position (2,0)
# we had two paths to the winning state, UURRR or RRUUR
# so we give a prob 0f 0.5 to go R and 0.5 to go up
# very similar to our transitions (probs)
policy = {
    (2,0) : {'U':0.5,'R':0.5},
    (1,0) : {'U':1},
    (0,0) : {'R':1},
    (0,1) : {'R':1},
    (0,2) : {'R':1},
    (1,2) : {'U':1},
    (2,1) : {'R':1},
    (2,2) : {'U':1},
    (2,3) : {'L':1}
}

In [7]:
g.print_policy(policy)

------------------
 {'R': 1} | {'R': 1} | {'R': 1} |   |
------------------
 {'U': 1} |   | {'U': 1} |   |
------------------
 {'U': 0.5, 'R': 0.5} | {'R': 1} | {'U': 1} | {'L': 1} |


In [8]:
# printing the policy is now pretty ugly
# one more thing for my future self to improve

In [9]:
# from here lets follow the same steps
# lets also calculate rewards r

def build_transition_probs_and_rewards(g):
    # probs_env : p(s'|s,a)
    # r : rewards @ s'
    probs = g.probs
    probs_env = {}
    r = {}
    for (s,a),next_state_2_probs in probs.items():
        for s_prime,prob in next_state_2_probs.items():
            probs_env[s_prime,s,a] = prob
            r[s_prime] = g.rewards.get(s_prime,0)
    return probs_env,r

In [10]:
p,r = build_transition_probs_and_rewards(g)

In [11]:
def build_action_probs(policy):
    # pi : pi(a|s)
    pi = {}
    for s,a2prob in policy.items():
        for a,prob in a2prob.items():
            pi[a,s] = prob
    return pi

In [12]:
pi = build_action_probs(policy)

In [13]:
# now we perform policy evaluation
def policy_eval(g):
    all_states = g.all_states()
    V = np.zeros((g.rows,g.cols))
    all_actions = ['U','R','D','L']
    delta = 0
    thresh = 1e-5
    gamma = 0.9
    
    i = 1
    while True:
        # this is just to calculate delta
        V_old = V.copy()
        for s in all_states:
            v_new = 0
            for a in all_actions:
                for s_prime in all_states:
                    v_new += pi.get((a,s),0)*p.get((s_prime,s,a),0)*(g.rewards.get(s_prime,0) + gamma*V[s_prime])
            V[s] = v_new
        delta = np.max(np.abs(V_old-V))
        print('iteration: ',i,' delta: ',delta)
        print('Values: ')
        g.print_values(V)
        print('')
        if delta < thresh:
            break
        i += 1
        V_old = V.copy()
    return V             

In [14]:
V = policy_eval(g)

iteration:  1  delta:  1.0
Values: 
---------------------------
 0.00| 0.00| 1.00| 0.00|
---------------------------
 0.00| 0.00|-0.05| 0.00|
---------------------------
 0.00| 0.00|-0.04|-0.04|

iteration:  2  delta:  0.9
Values: 
---------------------------
 0.00| 0.90| 1.00| 0.00|
---------------------------
 0.00| 0.00|-0.05| 0.00|
---------------------------
 0.00|-0.04|-0.04|-0.04|

iteration:  3  delta:  0.81
Values: 
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00|-0.05| 0.00|
---------------------------
 0.31|-0.04|-0.04|-0.04|

iteration:  4  delta:  0.0
Values: 
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00|-0.05| 0.00|
---------------------------
 0.31|-0.04|-0.04|-0.04|



In [16]:
# if we look at the value, we can see that it still converges quite fast
# the important part is making sure these values make sense
# In the previous script the values were symmetric for either paths to the goal
# this time they are not symmetric
# due to the fact that the state transition beside the losing state is now probabilistic
# we'll recall that even if we select the action to go up
# the enviroment dynamic dicdate that we still have a 50% chance of ending up in the losing state
# feel free to do the calculation on paper to check
# One thing that might seem strange is that, one step awawy from -0.05 is -0.04 but one step away from that -0.04
# we might think why doesnt this value decrease in magnitude since we have a gamma of 0.9
# in fact it does decrease but we dont see it since we are cutting of first 2 decimal places only

In [17]:
V

array([[ 0.81    ,  0.9     ,  1.      ,  0.      ],
       [ 0.729   ,  0.      , -0.05    ,  0.      ],
       [ 0.309825, -0.0405  , -0.045   , -0.0405  ]])

<h1>Math</h1>

In this section, we'll be continuing our discussion of dynamic programming

---

<h3>Dynamic Programming Continued</h3>

Let's recall again the two tasks that we care about in reinforcement learning 

Task number one is given a policy, tell me the value of that policy

Task number two is given an environment, tell me the best policy 

Note that we just solved task number one in the previous sections

So in this section, we'll start on solving task number two

The main principle we need for solving task number two is an idea called <strong>policy improvement</strong>

This answers the question, given a policy, how do we find a better policy?

We can imagine that if we've able to answer this question, then I've solved the problem

This is because if we can find a better policy, given an existing policy, then we can just keep iterating on our policy 

Since each step leads to a better policy we will have a monotonically increasing improvement in policies

So how does policy improvement work?

---

<h3>Policy Improvement</h3>

Let's begin with what we know so far and then just make a very tiny change

We'll start with a given policy $\pi$ 

From the previous lectures, we now know how to find its value function $V_\pi(s)$

Now, suppose that we just consider one single state $s$ 

And suppose that when we are in this state $s$, instead of following the policy, we decide to do something else

Suppose that we take some action $a$ which is not the same as the action we would have taken according to the policy $\pi$

Now how can we find the value of this action if we were to perform the action in this state and then follow our given policy thereafter?

Of course, this is what the action value $Q$ tells us

$Q_\pi(s,a)$ tells us the value of doing action $a$ while in state $s$ and then following the policy $\pi$ thereafter

Intuitively, we can see that if $Q_\pi(s,a) > V_\pi(s)$, then our return for this episode  is expected to be better than if we had just followed $\pi$ the whole time

The subtle thing to notice is that what we are talking about here is just a single action out of an entire series of actions

Imagine playing a game from start to end

As we recall, we call this an episode

Essentially, an episode is just a series of states, actions and rewards in a sequence

So we play one episode and on just this one action, we decide to deviate from our existing policy

The $Q$ function tells us that if the value for this action is greater than $V_\pi$, then making just this one change will have improved our expected return

---

<h3>How do we find a better action?</h3>

The next question to answer is this 

Suppose that we have some state $s$ and we are interested in finding some better action

We know that we can look at $Q$ to tell us whether or not a new action will be better than the current policy

So how can we find such an action?

Better yet, how can we find the best action?

Well, here's a simple solution

Why not just look through the $Q$ table over all  possible actions from this given state $s$ and then pick the one that gives us the maximum $Q$?

As we may recall, this is called the $\arg \max$

The best action to perform in the state would be 

$$\large a^* = \arg \max_a Q_\pi(s,a)$$

And again, this is just a change to one single action over an entire episode

And then we follow the prescribed policy $\pi$ thereafter

Let's call the action we chose $a^*$

So if $Q_\pi(s,a^*) > V_\pi(s)$ events than we have chosen the best action from the state $s$ that improves the expected return

---

<h3>A Subtly Different Question</h3>

So we have just seen how deviating from our policy by performing just one action differently during an episode can improve our expected return

The next question to consider is this

We know that it's possible to encounter the same state twice or more during an episode 

so we can ask a similar but subtly different question

What if we perform this other action not just once, but every time we visit that state?

In this case, we have created a new policy because the action we prescribed to this specific state is now different than what it was before

So let's call this new policy $\pi^\prime(s)$

Our original given policy was $\pi(s)$

Now, this is a very subtle difference from what we were discussing before

Previously, we considered the case where we changed the action only once and then followed the given policy thereafter

This is different because now we are saying every time we see the state $s$, we are going to perform a different action given by $\pi^\prime$

In this case, we do not follow the prescribed policy by thereafter

---

The next question to answer is this, is $\pi^\prime(s)$ a better policy than $\pi(s)$?

Now, this might seem obvious, but the reasoning is subtle and it's actually not obvious if we think about it

Suppose that we choose one single state to change the action

We look at $Q_\pi(s,a)$ and we take the $\arg \max$ over all actions $a$

Then we say, our new policy, $\pi^\prime$, will replace the existing action with this new action

Suppose that all other state action mappings in the policy remain the same

Previously we showed that if we only change the action once and follow the given policy thereafter, this leads to an improvement

This just follows directly from the Bellman equation for $Q$ 

$$\large Q_\pi(s,a) = \sum_{s^\prime}\sum_r p(s^\prime,r \vert s,a) \{r + \gamma V_\pi(s^\prime)\}$$

The right hand side contains $V_\pi$, which means that we have to follow the existing policy $\pi$ for this equation to hold

But now we are asking a slightly different question

Now the right hand side no longer applies because we are no longer following $\pi$, but we've created a new policy $\pi^\prime$

So hopefully we can see that, in fact, it's not obvious that the value for the new policy is better than that of the old policy

We actually do not have any equation telling us that this should be the case

Help : First we need to understand $Q$,  lets assume for simplicity that our enviroment is fully deterministic, we then express $Q$ as $Q(s,a) = r + \gamma V_\pi(s^\prime)$, this means that the expected sum of future rewards when in state $s$ and doing action $a$ and arriving at state $s^\prime$ is simply the reward we get for arriving at $s^\prime$ + the sum of the future rewards when we are in $s^\prime$, again since everything is deterministic no need to sum over $s^\prime$ or $r$

The idea is, for once we can ask, if we are in state $s$, which action $a^*$ gives us the highest value $Q_\pi(s,a)$, obviously this action is the best action to perform now

But if so, why dont we keep doing this forever?

The idea is that, when calculating $Q_\pi(s,a)$ to show that $a^*$ is the best action, we used $V_\pi(s^\prime)$, this means that we assumed that we will follow the poilicy $\pi$ once we have reached state $s^\prime$

So our deduction that $a^*$ is the best action is based on the assumption that we will follow $\pi$ thereafter, again this is mathematically manifested in the usage of $V_\pi$, now once we reach $s^\prime$ we have to abide by our assumption, so this result is only valid provided that we follow $\pi$ thereafter 

Its as if we have promised th follow $\pi$ thereafter to get $a^*$ in return

---

<h3>Policy Improvement Theorem</h3>

Luckily, it turns out to be true, the expected return for following $\pi^\prime$ is better than the expected return for following $\pi$

And note that when we say better than, we're just using casual words for $\ge$

And this is the case when we choose $\pi^\prime$ as described previously

Now, the proof of this is outside the scope of this notebook

This is one of those rare situations where we don't think it adds anything to the intuition and in fact, we think it takes away from it

But if we're curious, we are encouraged to think about it ourselves

Help : We know that we depend on $V_\pi$, intuitevly we can just follow $\pi$ to get the rewards as expected, once we reach state $s$ again, we know that the future rewards from this point are higher if we chose $a^*$, so we can rechose $a^*$

What this is called is the policy improvment theorem

---

<h3>Policy</h3>

So to state it more formally, the policy improvement theorem is this

It says that, suppose we choose some different policy $\pi^\prime(s)$ for a given state $s$

Remember that for now we are just considering one single state

Suppose that 

$$\large \text{If } Q_\pi(s,\pi^\prime(s)) \ge V_\pi(s)$$

So we're following the same process we described before 

We are choosing some different action, $\pi^\prime(s)$ of events thereafter we continue to follow the policy $\pi$

Thats why both $Q$ and $V$ are subscripts by $\pi$

The policy improvement theorem says that if this is true, then it is also true that 

$$\large \text{Then } V_{\pi^\prime} \ge V_\pi(s) \text{ for all } s \in S$$


And therefore $\pi^\prime$ is a better policy than $\pi$

So note the subtle distinction between these two statements

We're saying that if this action was better to change once for the state $s$, then it's also better to change it to this action every time we visit the state $s$

---

Also note that if we have a strict inequality when we change the action once, then we'll have a strict inequality when we change the action permanently

This means that if we find some action such that $Q_\pi$ is greater then but not equal to $N_\pi$, then updating our policy with this new action will lead to a new value function $V^\prime_\pi$ that is strictly greater than $V_\pi$ 

$$\large \text{If} Q_\pi(s,\pi^\prime(s)) \boxed{>} V_\pi(s)$$

$$\large \text{Then } V_{\pi^\prime} \boxed{>} V_\pi(s) \text{ for all } s \in S$$

note : the $\text{ for all } s \in S$ is for both lines

---

<h3>Policy Improvement</h3>

The next question to consider is this, up until now, we've been considering what happens if we change the action for some individual state $s$

But let's say we perform this process for every state $s$ in the states because this is the process of policy improvement

We are given some policy $\pi$ and we would like to improve it

Let's also assume we've computed $Q_\pi$ and $V_\pi$ for all states and all actions

We know how to do this because this is just policy evaluation

So basically, we can think of policy improvement in pseudocode form

$\text{Let S = set of all states, except terminal state}$

$\text{for } s \in S:$

$\qquad \pi^\prime(s) = \arg \max_a Q_\pi(s,a)$

$\text{for } s \in S:$

$\qquad \pi^\prime(s) = \arg \max_a \sum\limits_{s^\prime}\sum\limits_r p(s^\prime,r \vert s,a) \{r + \gamma V_\pi(s^\prime)\}$

First, we have a loop that iterates over all states in the state space except for the terminal state

Since we never take any actions from the terminal state, it is irrelevant 

Now for each of these actions, we take the $\arg \max$ over all actions of $Q_\pi(s,a)$

Now suppose that we like working with $V$ instead of $Q$ 

In this case we can replace $Q$ with the right hand side of the Bellman equation

OK, so this is the process of policy improvement

It basically means you improve the action for each state overall states

And by performing this process, we have improved our policy not just over a single state but overall states

---

<h3>Convergance</h3>

Let's now return to this concept of convergence

We can first recognise that by following the process of policy improvement, it's not possible for the policy to get worse

Therefore, the policy will always improve monotonically

Now, let's suppose that we reach a point where we perform the policy improvement process, and it turns out that the new policy is the same as the old policy, $\pi(s) = \pi^\prime(s)$, then that must mean that $V_\pi(s) = V_{\pi^\prime}(s)$

We can write this an equation form simply by plugging in what we had on the previous subsections for the policy improvement assignment

$$\large V_\pi(s) = \max_a \sum_{s^\prime} \sum_r p(s^\prime,r \vert s,a) \{r + \gamma V_\pi(s^\prime)\}$$

In this case, we are asking what happens when both sides are equal?

Well, you may recognize this from the previous notebooks

This is called the Bellman Optimality Equation

note : $V_\pi(s) \text{ is also called } V^*(s)$

So if we can reach this point where the Bellman optimality equation is satisfied, then we found the optimal policy

<h1>Math</h1>

In this section, we'll get one step closer to applying the policy improvement principle in practice

---

<h3>Policy Iteration</h3>

So far, we've learned how to improve a given policy

Interestingly, the input to this process is a policy, and the outputs of this process is also a policy $\pi^\prime = \text{PolicyImprovement}(\pi)$

The output policy happens to be better than the input policy

Now, consider what would happen if we applied the process again

Well, reason should tell us that the result would be an even better policy  $\pi^{\prime\prime} = \text{PolicyImprovement}(\pi)$

If we apply the process again an even better policy $\pi^{\prime\prime\prime} = \text{PolicyImprovement}(\pi)$

So this is the concept of policy iteration

---

Now, we have to zoom in a little bit, because there's one more step that we've almost forgotten about

The process of policy improvement we described in the previous section requires us to find the value function

Therefore, we know that policy evaluation should be part of this process

Specifically, the full process is this

Suppose we're given some initial random policy $\pi_0$

The next step is to evaluate this policy

So we find $V_{\pi0}$

As we recall, this is needed for the policy improvement step

Then we learned in the last section that we can improve the current policy by following the process of policy improvement 

This will give us $\pi_1$

In order to improve $\pi_1$, we must first find its value function

So we do another round of policy evaluation and this gives us $V_{\pi1}$

Clearly we can now improve this policy to get $\pi_2$ which is better than $\pi_1$

Then if we want to improve $\pi_2$, we have to find its value

So we evaluate $\pi_2$ and we get $V_{\pi_2}$

And obviously we can repeat this pattern until our policy stops changing

This is the process of policy iteration

$\text{Given: initial random policy } \pi_0$

$V_{\pi0} = \text{PolicyEvaluation}(\pi_0)$

$\pi_1 = \text{PolicyImprovement}(V_{\pi0})$

$V_{\pi1} = \text{PolicyEvaluation}(\pi_1)$

$\pi_{2} = \text{PolicyImprovement}(V_{\pi_1})$

<img src='extras/55.7.PNG' width='200'><img>

---

<h3>Policy Iteration Pseudocode</h3>

So we hope we'll agree that given what we've learned so far, the process of policy iteration is pretty straightforward 

It's nothing but the repetition of two previous concepts we already learned about

Nonetheless, it's still helpful to walk through some pseudocode before we move on to implementing this ourselves

Now, at a high level, this is probably pretty basic

We start by initializing some random policy

Then we have an infinite loop 

Inside this loop we simply have two steps

Step number one is to evaluate the current policy

Step number two is to perform policy improvement

At this point, we can check if our new policy is the same as our old policy

And if that's the case, we can exit the infinite loop

```
Initialise: random policy
Loop:
    V = evaluate policy
    policy = improve policy using V
    if new policy = old policy: break
```

---

<h3>Policy Iteration Pseudocode in Detail</h3>

Let's now zoom in so that we can consider this in more detail $\color{green}{\text{Policy Evaluation}},\color{blue}{\text{Policy Improvement}}$


$\text{Inititalise random policy } \pi \\ \text{Loop:} \\ \qquad \color{green}{\boxed{ \text{Initialise: V(s) = 0 or random (except terminal s, where V(s) = 0)} \\ \qquad  \text{Loop:} \\ \qquad  \qquad \Delta = 0 \\ \qquad  \qquad \text{for s in all_non_terminal_states:} \\ \qquad  \qquad \qquad v_{\text{old}} = V(s) \text{# store the existing value} \\ \qquad  \qquad \qquad V(s) = \sum_a \pi(a \vert s) \sum_{s^\prime} \sum_r p(s^\prime,r \vert s,a)[r + \gamma V(s^\prime)] \\ \qquad  \qquad \qquad \Delta = max(\Delta, \vert v_{\text{old}} - V(s) \vert) \\ \qquad  \qquad if \Delta < \text{threshold}: \\ \qquad  \qquad \qquad \text{break}}} \\ \color{blue}{\qquad \boxed{ \text{is_policy_stable = true} \\ \qquad \text{for s in all_non_terminal_states:}\\ \qquad \qquad a_{\text{old}} = \pi(s) \\ \qquad \qquad \pi(s) = \arg \max\limits_a \sum\limits_{s^\prime} \sum_r p(s^\prime,r \vert s,a)[r + \gamma V(s^\prime)] \\ \qquad \qquad if a_{\text{old}} \neq \pi(s): \text{is_policy_stable=false} \\ \qquad \text{if is_policy_stable}: \\ \qquad \qquad break}}$

Again, we start by initializing some random policy

Step number one is to evaluate this policy, so let's write this out explicitly

This is essentially just a copy of what we learned previously 

As before we start by initializing $V(s)$ and then proceeding with an infinite loop

Next, we create a variable $\Delta$ which will store the maximum change on each iteration

Then we loop through all the states in the state space except for the terminal state

Inside this loop, we save the old value of $V(s)$

Then we compute the new value of $V(s)$ using the Bellman equation

Then we update $\Delta$ using the absolute difference between the new $V(s)$ and the old $V(s)$

Outside the loop, we check whether or not Delta is less than our minimum threshold

If it is, then we can break out of the loop and policy evaluation is complete

Step number two, proceeds like this

First, we create a flag that tells us whether or not our policy has changed

We'll call this ```is_policy_stable```

The initial value for this is true, but if any of the subsequent steps lead to a change, we will set it to false

Next, we loop through all the states in the state space except for the terminal state 

Inside this loop, we save a copy of the action given by our existing policy for the state $s$

We'll call this ```a_old```

Then we perform policy improvement for the state using the ```arg max``` equation We learned previously.

Note that instead of using ```Q```, we use ```V```

This is because, as we recall, ```V``` is easier to compute than ```Q``` 

For ```V```, if we have `S` states, then we only need to hold it `S` values

But if we have `A` actions, then we need `S x A` values to hold `Q` 

So using `V` is a bit more memory efficient

Next, we check whether our improved action is the same as ```a_old```

If this is not the case, then we set our `is_policy_stable` flag to false

When we finish looping through all the states, we check whether or not there `is_policy_stable` flag has been set to false

If that was not the case and the flag is still true, then we know we can quit because the policy has not changed

Since the policy hasn't changed, the value will also not change

And of course, if the policy has changed, then training is not yet complete and we loop back around to perform both steps once again

---

<h3>Subtle Point</h3>

Now, there's one extra subtle point to consider

Recall that as we discussed before, optimal values are unique, but optimal policies are not unique

This is because if one value function is better than another value function, then by definition only one of them can possibly be optimal

On the other hand, two or more different policies can lead to the same value function

Therefore, neither is better than the other, they are both optimal

But what would happen if we performed our algorithm and we just kept switching back and forth between two distinct optimal policies?

In this case, the loop would never terminate

Therefore, we can add one extra condition for whether or not we should quit

Specifically, if the policy is stable, then we know it's OK to quit

But also if the value is stable, then we also know it's OK to quit

This allows us to recognize that it's possible for more than one policy to be optimal, but if we find that the value is no longer changing, then whatever policy we have found stable or not is an acceptable answer

---

<h3>Small Improvement</h3>

So there's one slight opportunity for improvement in the preceding pseudocode

Specifically, consider what will happen if the policy only changes by a small amount from one iteration to the next

Say only one or two actions have changed 

In this case the value for the new policy will not be that different from the value of the old policy

So it seems wasteful to start the policy evaluation process from scratch on each round

Instead, we would like our initialization point to be close to where we want to end up 

To that end, instead of initializing `V` to a bunch of zeros or random numbers each time, it's more efficient to save the current `V` and use that as our initial value for the next round of policy evaluation

In this way, we can utilize the fact that if `V` doesn't change that much, then policy evaluation will be completed in fewer steps

We swill test this out in code and observe this for ourselves for the next excercise

<h1>code</h1>

In [29]:
# lets write policy iteration code 
# we will be using the deterministic GridWorld enviroment

In [30]:
import numpy as np

In [31]:
# first lets implement our Gridworld enviroment
# we will be using the same interface as described above

class GridWorld:
    def __init__(self,rows,cols,start_pos):
        self.rows = rows
        self.cols = cols
        self.pos = start_pos
    
    def current_state(self):
        return self.pos
    
    def set_state(self,state):
        self.pos = state
        
    def set_actions_rewards(self,actions,rewards):
        self.actions = actions
        self.rewards = rewards
        
    def all_states(self,):
        return [(i,j) for i in range(self.rows) for j in range(self.cols)]
    
    def is_terminal(self,state):
        # we cant perform an action in a terminal state
        # so we expect not to find it in the keys
        return state not in self.actions.keys()
    
    def game_over(self):
        return self.is_terminal(self.pos)
    
    def next_state(self,s,a):
        # return next state should we take action a
        # but do not take that action
        i,j = s
        # if we are allowed to do this action in our current position
        if a in self.actions.get(s,[]): 
            if a == 'R':
                j += 1
            if a == 'L':
                j -= 1
            if a == 'U':
                i -= 1
            if a == 'D':
                i += 1
        return (i,j)
    
    def move(self,a):
        # return next state should we take action a
        # this time we actually make the move
        # and thus return reward
        i,j = self.pos
        # if we are allowed to do this action in our current position
        if a in self.actions.get(self.pos,[]): 
            if a == 'R':
                j += 1
            if a == 'L':
                j -= 1
            if a == 'U':
                i -= 1
            if a == 'D':
                i += 1
        self.pos = (i,j)
        # if no reward return 0
        return self.rewards.get(self.pos,0)
    
    # lets add methods to print values and actions
    def print_values(self,V):
        for i in range(self.rows):
            print('-'*9*self.rows)
            for j in range(self.cols):
                v = V[i,j]
                if v >= 0:
                    # if 0 or +ve, add space so it aligns well when we have -
                    print(" %.2f|" %v,end="")
                else:
                    print("%.2f|" %v,end="")
            print("")
    
    def print_policy(self,P):
        for i in range(self.rows):
            print('-'*6*self.rows)
            for j in range(self.cols):
                print(" %s |" %P.get((i,j),' '),end="")
            print("")
                
    def state_to_loc(self,state): # convert state tupe (0,1) to location 1
        return state[0]*self.rows + state[1]

In [32]:
def standrad_grid():
    g = GridWorld(3,4,(2,0))
    rewards = {(0,3):1,(1,3):-1}
    actions = {
        (0,0): ('D','R'),
        (0,1): ('L','R'),
        (0,2): ('L','D','R'),
        (1,0): ('U','D'),
        (1,2): ('U','D','R'),
        (2,0): ('U','R'),
        (2,1): ('L','R'),
        (2,2): ('L','R','U'),
        (2,3): ('L','U')
    }
    g.set_actions_rewards(actions,rewards)
    return g

In [33]:
g = standrad_grid()

In [34]:
# now lets build the transition probabilities
def build_transition_probs_rewards(g):
    # we want to calculate p(s'|s,a)
    # if there is an action a to take us from s to s'
    # then p(s' | s,a) is 1
    # else it is 0
    # recall that our enviroment is deterministic
    # for each action, there is only one state to go to
    all_states = g.all_states()
    all_actions = ['U','R','D','L']
    probs_env = {}
    r = {}
    for s in all_states:
        for a in all_actions:
            s_prime = g.next_state(s,a)
            probs_env[(s_prime,s,a)] = 1
        r[s] = g.rewards.get(s,0)

    
    return probs_env,r

In [35]:
p,r = build_transition_probs_rewards(g)

In [36]:
# build pi(a|s)
def build_action_probs(policy):
    all_states = g.all_states()
    state_space = len(all_states)
    pi = {}
    for s in all_states:
        if not g.is_terminal(s):
            a = policy.get(s)
            pi[a,s] = 1
    return pi

In [37]:
# lets follow the algorithm

# we initialise V[s] to zeros here
# since once we are inside we will be using the old values
# initialise V(s) = 0
all_states = g.all_states()
state_space = len(all_states)
V = np.zeros(state_space).reshape((g.rows,g.cols))


all_actions = ['D','U','L','R']
action_space = len(all_actions)
threshold = 1e-5
gamma = 0.9

# initialise random policy
policy = {s:all_actions[np.random.choice(action_space)] for s in all_states if not g.is_terminal(s)}


# loop
while True:
    # skip initialisation we already did that
    # next time we get here we will be using the old value
    
    # policy evaluation
    pi = build_action_probs(policy)
    while True:
        delta = 0
        V_old = V.copy()
        for s in all_states:
            # continue only for non-termimal states
            if g.is_terminal(s):
                continue
            v_new = 0
            for a in all_actions:
                for s_prime in all_states:
                    v_new += pi.get((a,s),0)*p.get((s_prime,s,a),0)*(r.get(s_prime,0)+gamma*V[s_prime])
            V[s] = v_new 
        delta = np.max(np.abs(V-V_old))
        if delta < threshold:
            break
            
    # policy improvement
    policy_old = policy.copy()
    for s in all_states:
        action_values = []
        if g.is_terminal(s):
            continue
        for a in all_actions:
            action_value = 0
            for s_prime in all_states:
                action_value += p.get((s_prime,s,a),0)*(r.get(s_prime,0)+gamma*V[s_prime])
            action_values.append(action_value)
        policy[s] = all_actions[np.argmax(action_values)]
    if policy == policy_old:
        break
            


In [38]:
policy,V

({(0, 0): 'R',
  (0, 1): 'R',
  (0, 2): 'R',
  (1, 0): 'U',
  (1, 2): 'U',
  (2, 0): 'U',
  (2, 1): 'R',
  (2, 2): 'U',
  (2, 3): 'L'},
 array([[0.81  , 0.9   , 1.    , 0.    ],
        [0.729 , 0.    , 0.9   , 0.    ],
        [0.6561, 0.729 , 0.81  , 0.729 ]]))

In [39]:
g.print_values(V)

---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.90| 0.00|
---------------------------
 0.66| 0.73| 0.81| 0.73|


In [40]:
g.print_policy(policy)

------------------
 R | R | R |   |
------------------
 U |   | U |   |
------------------
 U | R | U | L |


In [41]:
# note that our optimal policy chooses 'U' in the starting state
# of course it would be as valid to go 'R'
# since that would still lead to the same Optimal Value
# this is an example of how the optimal policy is not unique, while the optimal value is

<h1>Math</h1>

In this section, we are going to look at policy iteration for Windy Gridworld

We are going to use the same Windy Gridworld as we looked at previously with a state beside the losing state is windy

<img src='extras/55.8.PNG' width='400'></img>

This makes it so that even if we choose to go up, we may still end up being pushed to the right such that we land in a losing state

Before we look at the code, it will be useful to think about what we expect our algorithm to do 

In our previous scripts taking either path to the winning state would be an optimal choice

But now, because of the fact that going beside the losing state is more dangerous, we might expect that our algorithm tries to take us away from this state

---

<h3>Policy Iteration (Windy) in Code</h3>

One thing that's not immediately clear is this

What should the action be if we are right beside the Losing state?

Obviously, we might think the best choice is to simply not go there

But remember, this is reinforcement learning and not our mind

So we still have to pick an optimal action for every state

What's not immediately clear is whether it's better to simply go up because the winning state is closer and to take the risk

Or is it better to go down and away from the danger?

<img src='extras/55.9.PNG' width='300'></img>

---

Here's something else that will make this problem a little more complicated

What if we add an associated cost for taking a step anywhere in the environment?

In other words, suppose that for every state we land in that is not one of the terminal states, we get a negative reward

This should alter our agents behavior

Suppose the reward for going to any non terminal state is just zero

Then it's always good to go down when we are beside the losing state because this imposes no penalty and going down as a deterministic move

<img src='extras/55.10.PNG' width='300'></img>

If we choose down, we will always go down

Only going up is probabilistic

However, what happens when the reward becomes negative?

For example, if we get $-10$ reward for each step we take, it's probably better to just jump into the losing state, which only gives us a $-1$ reward

<img src='extras/55.11.PNG' width='300'></img>

However, those are two extremes

What would be interesting is to see how the policy changes for those in-between values

What if we set the reward at each state to be $-0.1$?

What about $-0.2$ or $0.5$?

These will be investigated in the next code section

---

<h3>windy_grid_penalized</h3>

Let's begin by extending our `windy_grid` helper function to `windy_grid_penalized`

This takes in a parameter called step cost, which will be the reward for going to any state that is not the terminal state

The terminal state rewards will remain as $+1$ and $-1$

Of course, this is very easy to implement, using our existing API

We simply need to extend our rewards dictionary

```python
def windy_grid_penalised(step_cost=-0.1):
    g = WindyGrid(3,4,(2,0))
    rewards = {
        (0,0): step_cost,
        (0,1): step_cost,
        (0,2): step_cost,
        ...,
        (0,3): 1,
        (1,3): -1    
    }
    actions = {...}
    probs = {...}
    g.set(rewards,actions,probs)
    return g
```

<h1>Code</h1>

In [84]:
import numpy as np

class WindyGridWorld:
    def __init__(self,rows,cols,start_pos):
        self.rows = rows
        self.cols = cols
        self.pos = start_pos
    
    def current_state(self):
        return self.pos
    
    def set_state(self,state):
        self.pos = state
        
    def set_actions_rewards_probs(self,actions,rewards,probs):
        self.actions = actions
        self.rewards = rewards
        # we need probs to know where to go after taking action
        self.probs = probs
        
    def all_states(self,):
        return [(i,j) for i in range(self.rows) for j in range(self.cols)]
    
    def is_terminal(self,state):
        # we cant perform an action in a terminal state
        # so we expect not to find it in the keys
        return state not in self.actions.keys()
    
    def game_over(self):
        return self.is_terminal(self.pos)
    
    def move(self,a):
        # this time we sample a next state
        next_state_2_probs = self.probs(self.pos,a)
        next_states = list(next_state_2_probs.keys())
        probs = list(next_state_2_probs.values())
        next_state = np.random.choice(next_states,p=probs)
        self.pos = next_state
        return self.rewards.get(self.pos,0)
    
    # lets add methods to print values and actions
    def print_values(self,V):
        for i in range(self.rows):
            print('-'*9*self.rows)
            for j in range(self.cols):
                v = V[i,j]
                if v >= 0:
                    # if 0 or +ve, add space so it aligns well when we have -
                    print(" %.2f|" %v,end="")
                else:
                    print("%.2f|" %v,end="")
            print("")
    
    def print_policy(self,P):
        for i in range(self.rows):
            print('-'*6*self.rows)
            for j in range(self.cols):
                print(" %s |" %P.get((i,j),' '),end="")
            print("")
                
    def state_to_loc(self,state): # convert state tupe (0,1) to location 1
        return state[0]*self.rows + state[1]


In [85]:
def windy_grid_penalised(step_cost=-0.1):
    g = WindyGridWorld(3,4,(2,0))
    rewards = {
        (0,3):1,
        (1,3):-1,
        (0,0):step_cost,
        (0,1):step_cost,
        (0,2):step_cost,
        (1,0):step_cost,
        (1,1):step_cost,
        (1,2):step_cost,
        (2,0):step_cost,
        (2,1):step_cost,
        (2,2):step_cost,
        (2,3):step_cost,
    }
    
    actions = {
        (0,0): ('D','R'),
        (0,1): ('L','R'),
        (0,2): ('L','D','R'),
        (1,0): ('U','D'),
        (1,2): ('U','D','R'),
        (2,0): ('U','R'),
        (2,1): ('L','R'),
        (2,2): ('L','R','U'),
        (2,3): ('L','U')
    }
    probs = {
        ((2, 0), 'U'): {(1, 0): 1.0},
        ((2, 0), 'D'): {(2, 0): 1.0},
        ((2, 0), 'L'): {(2, 0): 1.0},
        ((2, 0), 'R'): {(2, 1): 1.0},
        ((1, 0), 'U'): {(0, 0): 1.0},
        ((1, 0), 'D'): {(2, 0): 1.0},
        ((1, 0), 'L'): {(1, 0): 1.0},
        ((1, 0), 'R'): {(1, 0): 1.0},
        ((0, 0), 'U'): {(0, 0): 1.0},
        ((0, 0), 'D'): {(1, 0): 1.0},
        ((0, 0), 'L'): {(0, 0): 1.0},
        ((0, 0), 'R'): {(0, 1): 1.0},
        ((0, 1), 'U'): {(0, 1): 1.0},
        ((0, 1), 'D'): {(0, 1): 1.0},
        ((0, 1), 'L'): {(0, 0): 1.0},
        ((0, 1), 'R'): {(0, 2): 1.0},
        ((0, 2), 'U'): {(0, 2): 1.0},
        ((0, 2), 'D'): {(1, 2): 1.0},
        ((0, 2), 'L'): {(0, 1): 1.0},
        ((0, 2), 'R'): {(0, 3): 1.0},
        ((2, 1), 'U'): {(2, 1): 1.0},
        ((2, 1), 'D'): {(2, 1): 1.0},
        ((2, 1), 'L'): {(2, 0): 1.0},
        ((2, 1), 'R'): {(2, 2): 1.0},
        ((2, 2), 'U'): {(1, 2): 1.0},
        ((2, 2), 'D'): {(2, 2): 1.0},
        ((2, 2), 'L'): {(2, 1): 1.0},
        ((2, 2), 'R'): {(2, 3): 1.0},
        ((2, 3), 'U'): {(1, 3): 1.0},
        ((2, 3), 'D'): {(2, 3): 1.0},
        ((2, 3), 'L'): {(2, 2): 1.0},
        ((2, 3), 'R'): {(2, 3): 1.0},
        ((1, 2), 'U'): {(0, 2): 0.5, (1, 3): 0.5},
        ((1, 2), 'D'): {(2, 2): 1.0},
        ((1, 2), 'L'): {(1, 2): 1.0},
        ((1, 2), 'R'): {(1, 3): 1.0},
      }
    g.set_actions_rewards_probs(actions,rewards,probs)
    return g


In [86]:
# next is the same

In [87]:
# try with different costs
# a 0 cost is the normal GridWorld Enviroment
step_costs = [0,-0.1,-0.2,-0.4,-0.5,-2]
envs = [windy_grid_penalised(c) for c in step_costs]

In [88]:
def play_games():
    def build_transition_probs_and_rewards(g):
        # probs_env : p(s'|s,a)
        # r : rewards @ s'
        probs = g.probs
        probs_env = {}
        r = {}
        for (s,a),next_state_2_probs in probs.items():
            for s_prime,prob in next_state_2_probs.items():
                probs_env[s_prime,s,a] = prob
                r[s_prime] = g.rewards.get(s_prime,0)
        return probs_env,r
    
    # build pi(a|s)
    def build_action_probs(policy):
        all_states = g.all_states()
        state_space = len(all_states)
        pi = {}
        for s in all_states:
            if not g.is_terminal(s):
                a = policy.get(s)
                pi[a,s] = 1
        return pi


    for g,c in zip(envs,step_costs):
        print("step cost: ",c)
        # from here lets follow the same steps
        # lets also calculate rewards r


        p,r = build_transition_probs_and_rewards(g)


        # lets follow the algorithm

        # we initialise V[s] to zeros here
        # since once we are inside we will be using the old values
        # initialise V(s) = 0
        all_states = g.all_states()
        state_space = len(all_states)
        V = np.zeros(state_space).reshape((g.rows,g.cols))


        all_actions = ['D','U','L','R']
        action_space = len(all_actions)
        threshold = 1e-5
        gamma = 0.9

        # initialise random policy
        policy = {s:all_actions[np.random.choice(action_space)] for s in all_states if not g.is_terminal(s)}


        # loop
        while True:
            # skip initialisation we already did that
            # next time we get here we will be using the old value

            # policy evaluation
            pi = build_action_probs(policy)
            while True:
                delta = 0
                V_old = V.copy()
                for s in all_states:
                    # continue only for non-termimal states
                    if g.is_terminal(s):
                        continue
                    v_new = 0
                    for a in all_actions:
                        for s_prime in all_states:
                            v_new += pi.get((a,s),0)*p.get((s_prime,s,a),0)*(r.get(s_prime,0)+gamma*V[s_prime])
                    V[s] = v_new 
                delta = np.max(np.abs(V-V_old))
                if delta < threshold:
                    break

            # policy improvement
            policy_old = policy.copy()
            for s in all_states:
                action_values = []
                if g.is_terminal(s):
                    continue
                for a in all_actions:
                    action_value = 0
                    for s_prime in all_states:
                        action_value += p.get((s_prime,s,a),0)*(r.get(s_prime,0)+gamma*V[s_prime])
                    action_values.append(action_value)
                policy[s] = all_actions[np.argmax(action_values)]
            if policy == policy_old:
                break

        g.print_values(V)
        g.print_policy(policy)    

In [89]:
play_games()

step cost:  0
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.48| 0.00|
---------------------------
 0.66| 0.59| 0.53| 0.48|
------------------
 R | R | R |   |
------------------
 U |   | D |   |
------------------
 U | L | L | L |
step cost:  -0.1
---------------------------
 0.62| 0.80| 1.00| 0.00|
---------------------------
 0.46| 0.00|-0.04| 0.00|
---------------------------
 0.31| 0.18| 0.06|-0.04|
------------------
 R | R | R |   |
------------------
 U |   | D |   |
------------------
 U | L | L | L |
step cost:  -0.2
---------------------------
 0.43| 0.70| 1.00| 0.00|
---------------------------
 0.19| 0.00|-0.15| 0.00|
---------------------------
-0.03|-0.23|-0.34|-0.50|
------------------
 R | R | R |   |
------------------
 U |   | U |   |
------------------
 U | L | U | L |
step cost:  -0.4
---------------------------
 0.05| 0.50| 1.00| 0.00|
---------------------------
-0.36| 0.00|-0.25| 0.00|
---------------------------


now lets look at the results

`step_cost = 0` , so no penalty

As promised, The policy is to go away from the state right beside the losing state

We choose to go down because that action is deterministic and always leads to us going down

But going up would be dangerous because there's a chance we would end up in the losing state 

By going  down we guarantee that we will reach the winning state and thus the value for this state is positive

We might want to double check that the value goes down by a factor of $0.9$ for every step away from the goal state that we go

It starts at $1$, then $0.9$, then $0.81$, then $0.73$, then $0.66$ then $0.59$ then $0.53$, then $0.48$

---

`step_cost = -0.1`


As we can see, this death penalty is not enough to change our policy

However, the value of being in the state beside the losing state goes down to a negative number

---

`step_cost = -0.2`

This is where things get interesting

Now, when we are beside the losing state, we choose to go up instead of down

This is because going all the way around now costs more than the expected cost of simply going up

Or put another way, they expected return of going up is higher than the expected return of going all the way around

Pay attention to the threshold of this decision as well

If we are below the state, beside the losing state, we still choose to go up.

But if we are to the left of that state, which is under the wall, we choose to go around, even though the winning state is further away

---

`step_cost = -0.4`

So our optimal policy has changed again

Now, if we are in the state underneath the wall, we choose to take the direct path to the winning
state rather than going all the way around

The step cost is simply too high to justify taking a long way

---

`step_cost =-0.5`

So this is also interesting

If we look at the state below the losing state, notice that the policy is now to simply go directly into the losing state instead of going around and attempting to get to the winning state

---

`step_cost = -2` 

At this point, the step cost is so high that even when we are right beside the losing state, it's
better to just go into the losing state instead of trying to go up to get to the winning state

<h1>Math</h1>

In this section, we'll be continuing our discussion of how to solve task number two, which is given an environment, how do we find the best policy?

---

<h3>Recap</h3>

In the previous sections, we saw that it was possible by simply combining two key ideas

The first idea was policy evaluation

In order to improve a policy, we have to know how good that policy is

This is done through the value function and the value function is found by performing policy evaluation

The second key idea is policy improvements

We determine the simple method that guarantees we can achieve a policy as good or better than a given policy just as long as we know its value function

We observe that if we just perform these two operations repeatedly, we would improve the policy each time

---

<h3>Drawback of Policy Iteration</h3>

Now, there is one drawback to the method we just described 

To see this, consider that the policy improvement process is a loop

```
Loop until convergance: 
    # policy evaluation
    Loop until convergance:
        Find V(s)
    # policy improvement
    Loop over all non-terminal states:
        Find π(s)
```

We repeat this loop until convergance 

Inside this loop, we have two steps

The first step is policy evaluation, and the second step is policy improvement

The second step is relatively efficient as it only requires a single pass through every state

On the other hand, the first step policy evaluation is not efficient

In fact, this step requires another loop in which we must wait for conversions

Therefore, we actually have two nested loops, both of which could potentially last for a very long time as we wait for them to converge

So hopefully we can see why this process would be slow

---

<h3>2 Ideas for Improvement</h3>

So here are two ways that we can consider speeding up the process of policy iteration

First, note that the policy itself is not really needed

Recall that the policy is simply the $\arg \max$ of $Q$

$$\large \pi_{k+1}(s) = \arg \max_a \sum_{s^\prime}\sum_r p(s^\prime,r \vert s,a) \{r + \gamma v_k(s^\prime)\}$$

We can also state this as the policy is the $\arg \max$ of the right hand side of the Bellman equation

In this case, we can use $V$

However, note that on the subsequent step, when we want to evaluate the policy, we don't really care about the $\arg \max$ of this expression, we care about the actual value of this expression

$\large v_{k+1}(s) = \max_a \sum_{s^\prime}\sum_r p(s^\prime,r \vert s,a) \{r + \gamma v_k(s^\prime)\}$

In other words, if we just want the value, we can take the max instead of the $\arg max$ and forget about which action is being chosen altogether

---

Here's another key idea

We know that the policy evaluation step is problematic

This is because this inner loop can potentially last forever

We know that, practically speaking, we can cut this short by stopping when we reach some threshold for the largest change in $V(s)$

Previously, we learned about one way to improve this calculation in the context of policy iteration

It was that instead of initializing $V(s)$ randomly or to all zeros, we could simply start at whatever values were currently stored in $V(s)$

This speeds things up, since we know that $V(s)$ won't change that much from one policy to the next

The result of this is that policy evaluation converges in fewer steps

Here's one way we can extend this idea

Instead of waiting for $V(s)$ to converge, what if we simply stop the evaluation process after a few steps?

And what if we take this to the extreme?

What if we perform just a single step of policy evaluation?

We might have a hunch that this converges since both the policy and the value function are both moving towards some optimal point

<img src='extras/55.12.PNG' width='400'></img>

---

<h3>Value Iteration</h3>

To come up with our next algorithm, we just need to combine these two ideas 

To recap, here's what they are

Idea number one, instead of taking the $\arg \max$ and getting the action, just take the max and get the value directly

Idea number two, instead of going through many steps of policy evaluation, just do a single step

When we combine these two ideas together, we get an algorithm called Value iteration

---

<h3>Value Iteration</h3>

So value iteration works like this

$\text{Initialise V(s) randomly or to 0, except terminal state which must be 0 } \\ \text{Loop}: \\ \qquad \Delta = 0 \\ \qquad \text{for s in all_non_terminal_states:} \\ \qquad \qquad v_{\text{old}} = V(s) \ \ \# \text{ store the existing value} \\ \qquad \qquad V(s) = \max\limits_a \sum\limits_{s^\prime}\sum\limits_r p(s^\prime,r \vert s,a)[r + \gamma V(s^\prime)] \\ \qquad \qquad \Delta = max(\Delta, \vert v_{\text{old}} - V(s)\vert) \\ \qquad \text{if } \Delta < \text{threshold:} \\ \qquad \qquad \text{break} \\ \text{# We only need to find the policy once, after V* is found} \\\text{for s in all_non_terminal_states:} \\ \quad \pi^*(s) = \arg \max\limits_a \sum\limits_{s^\prime} \sum\limits_r p(s^\prime,r \vert s,a)[r + \gamma V^*(s^\prime)]$  

We can see that it's a pretty short algorithm

So we start by initializing $V(s)$ randomly, except for the terminal state, which must have a value of zero

Then we enter an infinite loop

Inside this loop, we create a variable called $\Delta$

This will store the maximum change for this iteration of the loop

We'll initialize this value to zero, and we'll update it for each state that we see

Next, we loop through all the states in our state space except for the terminal state whose value should be fixed at zero

Inside the second loop, we store the current value of the $V(s)$ inside a variable called a $v_{\text{old}}$

Next, we take the $\arg \max$ of this expression, which is the equivalent to taking the max over all actions of the $Q$ function for the given state

This gives us the new value for $V(s)$

Note that this is similar to what we do for policy improvement, except that instead of taking the $\arg \max$ we now just take the max directly and forget about the actual action we would have chosen

Next, we update $\Delta$ to be the max of the current $\Delta$ and the current absolut change in $V(s)$

Finally, we check whether or not Delta is less than our threshold for convergence

If it is, then we break out of the loop

Once we've completed this loop, we found the optimal value function $V^*$

In order to find the optimal policy $\pi^*$, we need to loop through each state one last time

Inside this loop, we set $\p^*$ to be the $\arg \max$ of the usual Bellman expression

---

<h3>Value Iteration: Things to notice</h3>

So here are some interesting facts to note about value iteration

Note how in this pseudocode, we end up with only one infinite loop that stops when $V(s)$ converges

This is better than before where we had two nested infinite loops 

As before, while this one loop could potentially go on forever, practically speaking, we quit when the maximum change in $V(s)$ drops below some threshold

Furthermore, note that we don't have to deal with the issue where more than one policy can lead to
the optimal value function

As we recall, for policy iteration, it's possible for us to flip flop between two different optimal
policies, since optimal policies are not unique 

Since we're now dealing with the value function which is unique, the knowing once you break out of the loop is easy

Finally, note the parallels between this and policy evaluation

Policy evaluation is nothing but treating the Bellman equation as an update rule

We take the right hand side and we assign it to the left hand side

Convergence is achieved when the right hand side is equal to the left hand side, which is a fixed point for the update rule

Similarly, value iteration is nothing but treating the Bellman optimality equation as an update rule

Again, we take the right hand side and assign it to the left hand side

We've reached convergence when the right hand side is equal to the left hand side

<h1>Code</h1>

In [1]:
# lets try value evaluation for windy gridworld
import numpy as np

class WindyGridWorld:
    def __init__(self,rows,cols,start_pos):
        self.rows = rows
        self.cols = cols
        self.pos = start_pos
    
    def current_state(self):
        return self.pos
    
    def set_state(self,state):
        self.pos = state
        
    def set_actions_rewards_probs(self,actions,rewards,probs):
        self.actions = actions
        self.rewards = rewards
        # we need probs to know where to go after taking action
        self.probs = probs
        
    def all_states(self,):
        return [(i,j) for i in range(self.rows) for j in range(self.cols)]
    
    def is_terminal(self,state):
        # we cant perform an action in a terminal state
        # so we expect not to find it in the keys
        return state not in self.actions.keys()
    
    def game_over(self):
        return self.is_terminal(self.pos)
    
    def move(self,a):
        # this time we sample a next state
        next_state_2_probs = self.probs(self.pos,a)
        next_states = list(next_state_2_probs.keys())
        probs = list(next_state_2_probs.values())
        next_state = np.random.choice(next_states,p=probs)
        self.pos = next_state
        return self.rewards.get(self.pos,0)
    
    # lets add methods to print values and actions
    def print_values(self,V):
        for i in range(self.rows):
            print('-'*9*self.rows)
            for j in range(self.cols):
                v = V[i,j]
                if v >= 0:
                    # if 0 or +ve, add space so it aligns well when we have -
                    print(" %.2f|" %v,end="")
                else:
                    print("%.2f|" %v,end="")
            print("")
    
    def print_policy(self,P):
        for i in range(self.rows):
            print('-'*6*self.rows)
            for j in range(self.cols):
                print(" %s |" %P.get((i,j),' '),end="")
            print("")
                
    def state_to_loc(self,state): # convert state tupe (0,1) to location 1
        return state[0]*self.rows + state[1]


In [2]:
def windy_grid():
    g = WindyGridWorld(3,4,(2,0))
    rewards = {(0,3):1,(1,3):-1}
    actions = {
        (0,0): ('D','R'),
        (0,1): ('L','R'),
        (0,2): ('L','D','R'),
        (1,0): ('U','D'),
        (1,2): ('U','D','R'),
        (2,0): ('U','R'),
        (2,1): ('L','R'),
        (2,2): ('L','R','U'),
        (2,3): ('L','U')
    }
    probs = {
        ((2, 0), 'U'): {(1, 0): 1.0},
        ((2, 0), 'D'): {(2, 0): 1.0},
        ((2, 0), 'L'): {(2, 0): 1.0},
        ((2, 0), 'R'): {(2, 1): 1.0},
        ((1, 0), 'U'): {(0, 0): 1.0},
        ((1, 0), 'D'): {(2, 0): 1.0},
        ((1, 0), 'L'): {(1, 0): 1.0},
        ((1, 0), 'R'): {(1, 0): 1.0},
        ((0, 0), 'U'): {(0, 0): 1.0},
        ((0, 0), 'D'): {(1, 0): 1.0},
        ((0, 0), 'L'): {(0, 0): 1.0},
        ((0, 0), 'R'): {(0, 1): 1.0},
        ((0, 1), 'U'): {(0, 1): 1.0},
        ((0, 1), 'D'): {(0, 1): 1.0},
        ((0, 1), 'L'): {(0, 0): 1.0},
        ((0, 1), 'R'): {(0, 2): 1.0},
        ((0, 2), 'U'): {(0, 2): 1.0},
        ((0, 2), 'D'): {(1, 2): 1.0},
        ((0, 2), 'L'): {(0, 1): 1.0},
        ((0, 2), 'R'): {(0, 3): 1.0},
        ((2, 1), 'U'): {(2, 1): 1.0},
        ((2, 1), 'D'): {(2, 1): 1.0},
        ((2, 1), 'L'): {(2, 0): 1.0},
        ((2, 1), 'R'): {(2, 2): 1.0},
        ((2, 2), 'U'): {(1, 2): 1.0},
        ((2, 2), 'D'): {(2, 2): 1.0},
        ((2, 2), 'L'): {(2, 1): 1.0},
        ((2, 2), 'R'): {(2, 3): 1.0},
        ((2, 3), 'U'): {(1, 3): 1.0},
        ((2, 3), 'D'): {(2, 3): 1.0},
        ((2, 3), 'L'): {(2, 2): 1.0},
        ((2, 3), 'R'): {(2, 3): 1.0},
        ((1, 2), 'U'): {(0, 2): 0.5, (1, 3): 0.5},
        ((1, 2), 'D'): {(2, 2): 1.0},
        ((1, 2), 'L'): {(1, 2): 1.0},
        ((1, 2), 'R'): {(1, 3): 1.0},
      }
    g.set_actions_rewards_probs(actions,rewards,probs)
    return g

In [6]:
def value_iteration():
    def build_transition_probs_and_rewards(g):
        # probs_env : p(s'|s,a)
        # r : rewards @ s'
        probs = g.probs
        probs_env = {}
        r = {}
        for (s,a),next_state_2_probs in probs.items():
            for s_prime,prob in next_state_2_probs.items():
                probs_env[s_prime,s,a] = prob
                r[s_prime] = g.rewards.get(s_prime,0)
        return probs_env,r
    
    # build pi(a|s)
    def build_action_probs(policy):
        all_states = g.all_states()
        state_space = len(all_states)
        pi = {}
        for s in all_states:
            if not g.is_terminal(s):
                a = policy.get(s)
                pi[a,s] = 1
        return pi

    # from here lets follow the same steps
    # lets also calculate rewards r


    p,r = build_transition_probs_and_rewards(g)


    # lets follow the algorithm

    # we initialise V[s] to zeros here
    # since once we are inside we will be using the old values
    # initialise V(s) = 0
    all_states = g.all_states()
    state_space = len(all_states)
    V = np.zeros(state_space).reshape((g.rows,g.cols))


    all_actions = ['D','U','L','R']
    action_space = len(all_actions)
    threshold = 1e-5
    gamma = 0.9

    # initialise random policy
    policy = {s:all_actions[np.random.choice(action_space)] for s in all_states if not g.is_terminal(s)}


    # get V*
    pi = build_action_probs(policy)
    while True:
        delta = 0
        V_old = V.copy()
        for s in all_states:
            # continue only for non-termimal states
            if g.is_terminal(s):
                continue
            action_values = []
            for a in all_actions:
                v_action = 0
                for s_prime in all_states:
                    v_action += p.get((s_prime,s,a),0)*(r.get(s_prime,0)+gamma*V[s_prime])
                action_values.append(v_action)
            V[s] = np.max(action_values)
        delta = np.max(np.abs(V-V_old))
        if delta < threshold:
            break

    # calculate policy only once at the end
    for s in all_states:
        action_values = []
        if g.is_terminal(s):
            continue
        for a in all_actions:
            action_value = 0
            for s_prime in all_states:
                action_value += p.get((s_prime,s,a),0)*(r.get(s_prime,0)+gamma*V[s_prime])
            action_values.append(action_value)
        policy[s] = all_actions[np.argmax(action_values)]

    g.print_values(V)
    g.print_policy(policy)    

In [7]:
g = windy_grid()

In [8]:
value_iteration()

---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00| 0.48| 0.00|
---------------------------
 0.66| 0.59| 0.53| 0.48|
------------------
 R | R | R |   |
------------------
 U |   | D |   |
------------------
 U | L | L | L |


In [108]:
# so we get the sane answer as with policy iteration
# However this is more effecient since we only need to do a single loop to find the optimal value
# And thereafter another single loop to find the optimal policy

<h1>Math</h1>

In this section, we'll summarize and review everything we learned in the notebook

---

<h3>Dynamic Programming</h3>

This notebook was all about how to solve two important problems in reinforcement learning

The previous notebook showed us the framework for reinforcement learning problems, the MDP 

Using the MDP, we were able to build on this framework to solve the problems of prediction and control

The prediction problem is a given a policy tell me the value of that policy

The control problem is given an environment, tell me the best policy

---

So to solve the prediction problem, we use policy evaluation

We learn that this is nothing but treating
the Bellman equation like an update rule

We noted that there are other solutions to this problem, such as using a basic linear solver

However, this is limited because it doesn't scale and it doesn't give us something to build on for later notebooks

In fact, we saw an example of that in this notebook

Specifically, value iteration is essentially just like policy evaluation, except that instead of doing a summation, we take the max

So even in this notebook, we've seen how the dynamic programming approach is more powerful than solving a linear system of equations 

To solve the control problem, we first learned about the principle of policy improvement

It was here that we learned about the policy improvement theorem

This theorem states that if changing an action  once improves the value for a given policy, then changing that action permanently for that state will lead to a better policy

Using this theorem, we developed the process of policy iteration

Policy iteration solves the control problem

We can start with a completely random policy and after running the process for long enough, we will end up with the optimal policy

One problem we discovered with policy iteration is that it can potentially be very slow

It has one infinite loop nested inside another infinite loop

We then considered a new approach called value iteration that just has a single loop

Essentially, value iteration is like policy iteration, except that we combine the evaluation step and the improvement step into a single operation

This converges much faster than policy iteration, and it doesn't require us to even store a policy at all

The policy is only computed after the value iteration loop is complete

---

<h3>But wait !!!</h3>

One interesting fact we may have noticed about this notebook is this 

Reinforcement learning is supposed to be all about learning from experience

Imagine a robot playing a game or solving a maze 

By playing the game or solving the maze a large number of times, it can use the rewards it achieved to learn the optimal policy

What was interesting about this notebook was that no actual games were played, so why did this happen?

In fact, no games were played because we had full knowledge about the dynamics of the environment

In other words, we knew as $p(s^\prime,r \vert s,a)$

The lesson is that, when we know this probability distribution, we don't actually need to play the game

The solution is just a matter of applying the Bellman equation or the Bellman optimality equation

So is learning from experience unnecessary?

The answer is no

This is because in the real world, we don't know this probability distribution

Imagine driving a car or playing a game of chess 

When we started the reinforcement learning series, we probably had in our minds this idea that we were going to teach and agents who play a game or solve some task 

We probably had in your mind that your program would learn from experience and not just use
probabilities

Now, of course, this is necessary in order to understand the next notebooks

In the next notebook, we will consider the more realistic scenario where this distribution is not known

In this case, we must learn from experience

Finally, note that we have different names for these approaches 

When we know the transition probabilities, that is our model of the environment 

When we use this model to find the solution, we call this a model based approach

When we learn from experience, we don't need a model of the environment, wo we call that a model free approach

The few following notebooks, will focus on model free approaches 

In more advanced notebooks, we might go back to model based approaches or even combine these two approaches to get a hybrid approach